# Data Science 4 : Séries temporelles (times series)

Enseignant : Jean Delpech

Cours : Data Science

Classe : M1 Data/IA

Année scolaire : 2025/2026

Dernière mise à jour : janvier 2026

## Module Data Science M1 - Séances 11 & 12

**Objectifs du cours :**
- Comprendre les concepts fondamentaux des séries temporelles
- Maîtriser la stationnarité et ses tests
- Implémenter et évaluer des modèles ARIMA
- Étendre aux modèles SARIMA pour les séries saisonnières

## Remarques préliminaires

Comme les images, les séries temporelles sont des données assez particulières, qui demandent un traitement et une approche spécifique (par rapport à une donnée qualitative ou numérique simple). C’est la raison pourquoi on les aborde dans ce module de data science, afin de vous familiariser avec la méthodologie employée pour manipuler et analyser ce type de données.

C’est un type de données avec une logique assez singulière. Elles sont issues de mesures réalisées à intervalle régulier, par exemple un relevé de température prise toutes les heures au même endroit (station météo d’une ville), ou encore le chiffre d’affaire réalisé chaque jour à la fermeture par une enseigne, le débit horaire d’un cours d’eau, ou enfin le cours d’une action en bourse. La caractéristique principale de ces données liée à une temporalité est que leur ordre est important, en fait ces données sont liées les unes aux autres (la valeur d’une mesure à un instant $t$ est lié à la valeur de la même mesure à l’instant $t-1$, $t-2$, etc.). C’est d’ailleurs ce que l’on va chercher à prédire : la température à partir des températures passées, le cours de bourse à partir du cours passé, le débit d’un cours d’eau à partir du débit passé, etc. Mais dans un contexte où les valeurs observées sont fortement corrélées les unes avec les autres – la valeur à $t$ dépend de (= est corrélée) la mesure à $t-1$ – les modèles utilisés classiquement (régression, etc.) ne sont pas suffisants pour être utilisés tels quels et réaliser des prédictions sur ce type de données.

Les modèles spécifiques aux séries temporelles reposent sur des opérations de bases et des concept tout aussi spécifiques de ce champ d’analyse : décomposition (tendance, saisonnalité, etc.), sationnarité et enfin auto-corrélation. Ces mots doivent vous faire sentir que nous nous lançons dans l’exploration d’un monde qui a ses propres règles. L’objectif de ce cours est avant tout de vous équiper des concepts qui guident les analyses, le prétraitement et la création de modèles. Nous apprendrons à utiliser la bibliothèque `statsmodels` pour l’analyse des séries temporelles, mais vous aurez tout le loisir dans les autres modules et par votre travail personnel d’approfondir l’usage des bibliothèques python. Profitez donc du cours en présentiel pour poser toutes les questions liées à la compréhension de la démarche plutôt qu’au code !

## Imports

In [ ]:
# Installation des bibliothèques (si nécessaire)
# !pip install numpy pandas matplotlib seaborn statsmodels yfinance

In [ ]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliothèques pour les séries temporelles
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.datasets import get_rdataset

# pour le calcul de métriques
from sklearn.metrics import mean_squared_error, mean_absolute_error

## 1. Introduction aux Séries Temporelles

### 1.1 Qu'est-ce qu'une série temporelle ?

Une **série temporelle** est une séquence de données observées à des intervalles de temps réguliers.

#### Définition formelle

Une série temporelle est une suite d'observations $(y_1, y_2, ..., y_T)$ indexées par le temps $t = 1, 2, ..., T$.

#### Caractéristiques principales

1. **Ordre temporel** : l'ordre des observations est crucial (contrairement aux données tabulaires classiques)
2. **Dépendance temporelle** : les valeurs successives sont souvent corrélées
3. **Intervalles réguliers** : les observations sont espacées de manière uniforme

#### Exemples d'applications

- **Finance** : cours boursiers, taux de change, prix des matières premières
- **Météorologie** : températures, précipitations, pression atmosphérique
- **Ventes** : chiffre d'affaires mensuel, demande de produits
- **Santé** : nombre de patients, épidémiologie
- **Transport** : trafic routier, nombre de passagers
- **Énergie** : consommation électrique, production renouvelable

### 1.2 Les composantes d'une série temporelle

Une série temporelle peut être décomposée en plusieurs composantes :

#### 1.2.1 **Tendance (Trend)** - $T_t$
- Mouvement à long terme de la série
- Croissante, décroissante ou stable
- Exemple : augmentation progressive des ventes d'une entreprise

##### Construction d’un exemple d’illustration :

In [ ]:
# Création d'une série temporelle simple avec Pandas
dates = pd.date_range(start='2020-01-01', end='2023-12-31', freq='D')
np.random.seed(42)

# Série avec tendance et bruit
tendance = np.linspace(100, 200, len(dates))
bruit = np.random.normal(0, 10, len(dates))
valeurs = tendance + bruit

serie_simple = pd.Series(valeurs, index=dates, name='Valeur')

# Affichage des premières valeurs
print("Premières valeurs de la série :")
print(serie_simple.head(10))
print(f"\nNombre d'observations : {len(serie_simple)}")
print(f"Période : de {serie_simple.index.min()} à {serie_simple.index.max()}")

In [ ]:
# Visualisation
plt.figure(figsize=(14, 5))
plt.plot(serie_simple, linewidth=1, alpha=0.8)
plt.title('Exemple de Série Temporelle avec Tendance', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Valeur', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

##### Exemple (données rééelles) :


#### 1.2.2 **Saisonnalité (Seasonality)** - $S_t$
- Variations régulières qui se répètent à intervalles fixes
- Période : annuelle, trimestrielle, mensuelle, hebdomadaire, journalière
- Exemple : pics de ventes en décembre (Noël)

##### Construction d’un exemple d’illustration

In [ ]:
# Création de données de température simulées (journalières)
np.random.seed(42)
dates_temp = pd.date_range(start='2018-01-01', end='2023-12-31', freq='D')

# Température avec composantes saisonnières
jours = np.arange(len(dates_temp))
temperature_moyenne = 15  # Température moyenne
amplitude_saisonniere = 10  # Amplitude de variation saisonnière
saisonnalite = amplitude_saisonniere * np.sin(2 * np.pi * jours / 365.25)
tendance_temp = 0.002 * jours  # Légère tendance au réchauffement
bruit_temp = np.random.normal(0, 3, len(dates_temp))

temperatures = temperature_moyenne + saisonnalite + tendance_temp + bruit_temp
serie_temp = pd.Series(temperatures, index=dates_temp, name='Température (°C)')

print("Statistiques descriptives :")
print(serie_temp.describe())

In [ ]:
# Visualisation des températures
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Série complète
axes[0].plot(serie_temp, linewidth=0.8, alpha=0.7)
axes[0].set_title('Température Journalière (2018-2023)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Température (°C)', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Zoom sur une année
serie_2022 = serie_temp['2022']
axes[1].plot(serie_2022, linewidth=1.5, color='orangered')
axes[1].set_title('Zoom sur l\'année 2022', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_ylabel('Température (°C)', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

##### Exemple (données rééelles) :


#### 1.2.3 **Cycle (Cyclical)** - $C_t$
- Fluctuations à long terme sans période fixe
- Différent de la saisonnalité (pas de période régulière)
- Exemple : cycles économiques (expansion/récession)

##### Exemple : cycle économique

#### 1.2.4 **Résidu / Bruit (Residual/Noise)** - $\epsilon_t$
- Fluctuations aléatoires inemxpliquées
- Ce qui reste après avoir enlevé tendance, saisonnalité et cycle

### 1.3 Modèles de décomposition

**Modèle additif** : $Y_t = T_t + S_t + \epsilon_t$
- Utilisé quand l'amplitude de la saisonnalité est constante

**Modèle multiplicatif** : $Y_t = T_t \times S_t \times \epsilon_t$
- Utilisé quand l'amplitude de la saisonnalité varie avec le niveau de la série

#### Exemple de création de série avec des composantes identifiables 

In [ ]:
# Création d'une série temporelle avec composantes distinctes
np.random.seed(42)
dates_ventes = pd.date_range(start='2018-01-01', end='2023-12-31', freq='MS')  # Début de mois
n = len(dates_ventes)

# 1. Tendance : croissance linéaire
tendance = np.linspace(1000, 2000, n)

# 2. Saisonnalité : pic en décembre (Noël), creux en février
mois = np.array([date.month for date in dates_ventes])
saisonnalite = 200 * np.sin(2 * np.pi * mois / 12) + 150 * (mois == 12)

# 3. Résidu : bruit aléatoire
residus = np.random.normal(0, 50, n)

# Série complète (modèle additif)
ventes = tendance + saisonnalite + residus
serie_ventes = pd.Series(ventes, index=dates_ventes, name='Ventes mensuelles')

print(f"Série de ventes mensuelles créée : {len(serie_ventes)} observations")
print(f"Période : {serie_ventes.index.min()} à {serie_ventes.index.max()}")

In [ ]:
# Visualisation de la série et de ses composantes
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Série complète
axes[0].plot(serie_ventes, linewidth=2, color='navy')
axes[0].set_title('Série Complète (Ventes Mensuelles)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Ventes', fontsize=11)
axes[0].grid(True, alpha=0.3)

# Tendance
axes[1].plot(dates_ventes, tendance, linewidth=2, color='green', label='Tendance')
axes[1].set_title('Composante : Tendance', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Tendance', fontsize=11)
axes[1].grid(True, alpha=0.3)

# Saisonnalité
axes[2].plot(dates_ventes, saisonnalite, linewidth=2, color='orange', label='Saisonnalité')
axes[2].set_title('Composante : Saisonnalité', fontsize=14, fontweight='bold')
axes[2].set_ylabel('Saisonnalité', fontsize=11)
axes[2].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[2].grid(True, alpha=0.3)

# Résidus
axes[3].plot(dates_ventes, residus, linewidth=1, color='red', alpha=0.7, label='Résidus')
axes[3].set_title('Composante : Résidus (Bruit)', fontsize=14, fontweight='bold')
axes[3].set_xlabel('Date', fontsize=11)
axes[3].set_ylabel('Résidus', fontsize=11)
axes[3].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 1.4 Décomposition automatique avec statsmodels

La bibliothèque `statsmodels` propose une fonction `seasonal_decompose` qui décompose automatiquement une série temporelle.

In [ ]:
# Décomposition de la série de ventes
decomposition = seasonal_decompose(serie_ventes, model='additive', period=12)

# Visualisation de la décomposition
fig, axes = plt.subplots(4, 1, figsize=(14, 12))

# Série observée
decomposition.observed.plot(ax=axes[0], color='navy', linewidth=2)
axes[0].set_ylabel('Observé', fontsize=11)
axes[0].set_title('Décomposition de la Série Temporelle', fontsize=16, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Tendance
decomposition.trend.plot(ax=axes[1], color='green', linewidth=2)
axes[1].set_ylabel('Tendance', fontsize=11)
axes[1].grid(True, alpha=0.3)

# Saisonnalité
decomposition.seasonal.plot(ax=axes[2], color='orange', linewidth=2)
axes[2].set_ylabel('Saisonnalité', fontsize=11)
axes[2].grid(True, alpha=0.3)

# Résidus
decomposition.resid.plot(ax=axes[3], color='red', linewidth=1, alpha=0.7)
axes[3].set_ylabel('Résidus', fontsize=11)
axes[3].set_xlabel('Date', fontsize=11)
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### Important !!!

1. **La décomposition permet de mieux comprendre la structure de la série**
2. **Le choix entre modèle additif et multiplicatif dépend des données**
3. **Le paramètre `period` doit correspondre à la fréquence de la saisonnalité** (12 pour mensuel avec saisonnalité annuelle)

### EXERCICE 1 : Analyse exploratoire d'une série temporelle

**Objectif** : Créer, visualiser et décomposer une série temporelle

**Consigne** : 
1. Créez une série temporelle mensuelle de consommation électrique sur 5 ans (2019-2023)
2. La série doit avoir :
   - Une tendance croissante (augmentation de 5% par an)
   - Une saisonnalité annuelle (pics en été et hiver, creux au printemps/automne)
   - Du bruit aléatoire
3. Visualisez la série
4. Décomposez-la avec `seasonal_decompose`
5. Analysez chaque composante (vérifiez qu’on retrouve bien les valeurs qui ont servi à créer la série).
6. Testez différentes valeurs du paramètre `period`. Que se passe-t-il ?

In [ ]:
# EXERCICE 1 - VPTRE CODE !

# 1. Création des dates
# dates_elec = ...

# 2. Création des composantes
# tendance_elec = ...
# saisonnalite_elec = ...
# bruit_elec = ...

# 3. Série complète
# serie_elec = ...

# 4. Visualisation
# ...

# 5. Décomposition
# decomposition_elec = ...

## 2. Stationnarité

### 2.1 Définition de la stationnarité

Une série temporelle est dite **stationnaire** si ses propriétés statistiques ne changent pas au cours du temps.

#### Stationnarité faible (ou au second ordre)

Une série $\{Y_t\}$ est stationnaire au second ordre si :

1. **Moyenne constante** : $E[Y_t] = \mu$ pour tout $t$
2. **Variance constante** : $Var(Y_t) = \sigma^2$ pour tout $t$
3. **Covariance dépendant uniquement du décalage** : $Cov(Y_t, Y_{t+h}) = \gamma(h)$ (ne dépend que de $h$, pas de $t$)

#### Pourquoi la stationnarité est-elle importante ?

- Une série stationnaire a des propriétés statistiques prévisibles
- Cette prévisibilité permet de construire des modèles ARIMA que nous allons utiliser pour analyser une certaine catégorie de séries dont une propriété est d’être **stationnaires** (il faut donc être capable de déterminer si la série remplie cette condition ou non)
- Les inférences statistiques sont plus fiables sur des données stationnaires

#### Séries non-stationnaires typiques

- **Tendance** : moyenne qui évolue dans le temps
- **Saisonnalité** : patterns réguliers qui se répètent
- **Variance changeante** : hétéroscédasticité (ex : volatilité croissante)
- **Marche aléatoire** (Random Walk) : $Y_t = Y_{t-1} + \epsilon_t$

### 2.2 Exemples de séries stationnaires et non-stationnaires

In [ ]:
# Création de différentes séries pour illustrer la stationnarité
np.random.seed(42)
n = 500

# 1. Bruit blanc (White Noise) - STATIONNAIRE
bruit_blanc = np.random.normal(0, 1, n)

# 2. Série avec tendance - NON STATIONNAIRE
tendance_lineaire = np.linspace(0, 10, n) + np.random.normal(0, 0.5, n)

# 3. Marche aléatoire (Random Walk) - NON STATIONNAIRE
marche_aleatoire = np.cumsum(np.random.normal(0, 1, n))

# 4. Processus AR(1) stationnaire : Y_t = 0.5 * Y_{t-1} + epsilon_t
ar1_stationnaire = [0]
for i in range(1, n):
    ar1_stationnaire.append(0.5 * ar1_stationnaire[-1] + np.random.normal(0, 1))
ar1_stationnaire = np.array(ar1_stationnaire)

In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Comparaison : Séries Stationnaires vs Non-Stationnaires', 
             fontsize=16, fontweight='bold', y=0.995)

# Bruit blanc - Stationnaire
axes[0, 0].plot(bruit_blanc, linewidth=1, color='green', alpha=0.7)
axes[0, 0].set_title('Bruit Blanc (Stationnaire)', fontsize=13, fontweight='bold', color='green')
axes[0, 0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[0, 0].set_ylabel('Valeur', fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# Tendance - Non stationnaire
axes[0, 1].plot(tendance_lineaire, linewidth=1, color='red', alpha=0.7)
axes[0, 1].set_title('Série avec Tendance (Non-Stationnaire)', fontsize=13, fontweight='bold', color='red')
axes[0, 1].set_ylabel('Valeur', fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# Marche aléatoire - Non stationnaire
axes[1, 0].plot(marche_aleatoire, linewidth=1, color='red', alpha=0.7)
axes[1, 0].set_title('Marche Aléatoire (Non-Stationnaire)', fontsize=13, fontweight='bold', color='red')
axes[1, 0].set_xlabel('Temps', fontsize=11)
axes[1, 0].set_ylabel('Valeur', fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# AR(1) - Stationnaire
axes[1, 1].plot(ar1_stationnaire, linewidth=1, color='green', alpha=0.7)
axes[1, 1].set_title('Processus AR(1) avec φ=0.5 (Stationnaire)', fontsize=13, fontweight='bold', color='green')
axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[1, 1].set_xlabel('Temps', fontsize=11)
axes[1, 1].set_ylabel('Valeur', fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### Récapitulatif

**Séries stationnaires** (bruit blanc, AR(1) avec |φ| < 1) :
- Oscillent autour d'une moyenne constante
- Variance stable
- Pas de tendance visible

**Séries non-stationnaires** (tendance, marche aléatoire) :
- Moyenne qui évolue dans le temps
- Peuvent s'éloigner indéfiniment de leur point de départ
- Pattern de "dérive" visible

Comment teste-t-on l’hypothèse de stationarité ? (on ne peut évidemment pas se contenter de juger « à l’œil »)

### 2.3 Test de stationnarité : Test ADF (Augmented Dickey-Fuller)

#### Principe du test ADF

##### Version courte :

Le test ADF teste l'hypothèse nulle de **présence d'une racine unitaire** (série non-stationnaire).

**Hypothèses** :
- $H_0$ : La série possède une racine unitaire (non-stationnaire)
- $H_1$ : La série est stationnaire

**Interprétation** :
- Si la **p-value < 0.05** : on rejette $H_0$ → la série est **stationnaire**
- Si la **p-value ≥ 0.05** : on ne peut pas rejeter $H_0$ → la série est **non-stationnaire**

##### Version détaillée (si vous voulez comprendre) :


**Qu'est-ce qu'une racine unitaire ?**

Avant d'expliquer le test ADF, il est essentiel de comprendre le concept de **racine unitaire**.
Mais auparavant, revenons au concept de processus autorégressif.

*Définition mathématique*

Reprenons la définition d’un [processus autorégressif](https://fr.wikipedia.org/wiki/Processus_autor%C3%A9gressif) d’ordre $p$ :

$$
AR(p): Y_t = c + \phi_1 Y_{t-1} + \phi_2 Y_{t-2}  + … + \phi_p Y_{t-p} + \varepsilon_t
$$

où $\varepsilon_t$ est un bruit blanc.

Considérons un processus autorégressif simple d'ordre 1 (noté AR(1)). Il s’agit d’un processus qui utilise seulement la valeur précédente (t-1) pour prédire la valeur actuelle (t) :

$$Y_t = c + \phi Y_{t-1} + \varepsilon_t$$

La valeur de $\phi$ est capitale : elle va déterminer le comportement de la série,

$$
\begin{cases} 
|\rho| <1 & \text{Processus stationnaire}\\
|\rho| =1 &  \text{Processus non-stationnaire : marche aléatoire}\\
|\rho| >1 & \text{Processus non-stationnaire (explosif)}\end{cases}
$$

Pourquoi ?

Imaginons la progression de la série après plusieurs séquences :

$$
Y_t = \phi Y_{t−1} + \varepsilon_t \\
Y_t = \phi (\phi Y_{t−2} + \varepsilon_{t-1}) + \varepsilon_t \\
Y_t = \phi (\phi (\phi (Y_{t−3} + \varepsilon_{t-2}) + \varepsilon_{t-1}) + \varepsilon_t \\
Y_t = \phi^3 Y_{t−3} + \phi^2 \varepsilon_{t-2} + \phi \varepsilon_{t-1} + \varepsilon_t
$$

On voit le motif qui se dessine et suggère la formule pour toute séquence depuis une origine $t=0$ :
$$
Y_t = \phi^t Y_0 \sum_{i=0}+^{t-1} \phi^i \varepsilon_{t-i}
$$

Qui montre immédiatement la nature exponentielle du phénomène :

* Si $\phi < 1$, on constate que le terme $\phi^t Y_0
 \rightarrow 0$ quand $t \rightarrow \infty$ (car $|\phi| < 1$), donc l'influence de la condition initiale $Y_0$ disparaît avec le temps (c’est ce que l’on appelle la partie transitoire). Il ne reste que l'influence du bruit blanc (ce que l’on appelle la partie permanente). Mais ATTENTION, ça ne veut pas dire que la relation $Y_t = \phi Y_{t−1} + \varepsilon_t$ s’arrête, cette loi est toujours valable à chaque instant. Par contre, cela veut dire que l’influence à long terme des séquences passées s’évanouit exponentiellement : seuls le passé (très) récent a réellement une influence. On peut démontre (mais cela demande des caculs et des concepts en probabilité/statistiques un peu plus avancés) que dans ce cas la variance est constante (ne dépend pas de $t$), que la moyenne est aussi constante, et que si la série s’éloigne de cette moyenne, il y aura comme une « force de rappel » qui l’y ramènera. 

* Si $\phi$ > 1 on a bien une explosion de la valeur de la série avec le temps. En effet, supposons que $/phi=1,2$ :
    * si $t=1, \phi^1 = 1,2$
    * si $t=10, \phi^10 \approx 6,2$
    * si $t=20, \phi^20 \approx 38$
    * si $t=50, \phi^50 \approx 9100$ 
    avec une telle évolution, il est évident que cette série ne saurait être stationnaire. De plus, la variance tend aussi très rapidement vers l’infini (évolution proportionnelle à $\phi^2t$). (Cf. [la page wikipedia](https://fr.wikipedia.org/wiki/Processus_autor%C3%A9gressif) pour la formule de la variance)

* Si $\phi = 1$, on dit que la série possède une **racine unitaire**.

    Dans ce cas particulier, le processus devient :

    $$Y_t = Y_{t-1} + \varepsilon_t$$

    C'est ce qu'on appelle une **marche aléatoire continue** (*continuous random walk*).
    Nous avons vu précédemment qu’un tel processus n’est pas stationnaire non plus.

>    *Pourquoi parle-t-on de "racine unitaire" ?*
>
>    Le terme vient de l'analyse de l'équation caractéristique du processus AR(1). On peut écrire l’équation d’un processus autorégressif en faisant appel à un opérateur de retard (*lag operator*) :
>$$
AR(p): Y_t = c + \phi_1 Y_{t-1} + \phi_2 Y_{t-2}  + … + \phi_p Y_{t-p} + \varepsilon_t \\
(1- \varphi_1 L-\varphi_2 L^2-\ldots-\varphi_p L^p) Y_t = c + \varepsilon_t 
$$
>
>En définissant l’opérateur de retard  ainsi: $L Y_t = Y_{t-1}$ >
>
>Cet opérateur permet d’écrire simplement :
>$$
L^2 Y_t = L(L Y_t) = L Y_{t-1} = Y_{t-2} \\
L^3 Y_t = Y_{t-3}
$$
>etc.
>  
>Dans le cas d’un $AR(1)$, on obtient :
>
>$$
Y_t = \phi Y_{t−1} + \varepsilon_t \\
Y_t =\phi L Y_t + \varepsilon_t \\
Y_t - \phi L Y_t = \varepsilon_t \\
(1 - \phi L) Y_t = \varepsilon_t
$$
>
>En procédant ainsi, on a écrit le processus sous une forme polynomiale.
>Le polynôme caractéristique associé à $(1−\phi L)$ est $\Phi(z) = 1 - \phi z$. L'équation caractéristique qui en découle est : $1 - \phi z = 0$, soit $z = \frac{1}{\phi}$
>
>Si $\phi = 1$, alors $z = 1$ : la racine de l'équation caractéristique est égale à 1, d'où le terme **racine unitaire**.

#### Conséquences de la présence d'une racine unitaire

**1. Non-stationnarité**

Si $\phi = 1$, le processus devient :

$$Y_t = Y_{t-1} + \epsilon_t = Y_0 + \sum_{i=1}^{t} \epsilon_i$$

C'est la somme cumulée des chocs aléatoires depuis l'origine (on par le de « chocs » dans une marche aléatoire – cf. mouvements browniens – il s’agit des changements de directions aléatoires dans la marche à chaque étape).

- **Variance** : $Var(Y_t) = t \sigma^2$ → elle **augmente avec le temps** (non-stationnaire !)
- **Moyenne** : dépend de $Y_0$ et des chocs passés
- **Mémoire infinie** : un choc $\epsilon_t$ affecte **toutes** les valeurs futures de manière permanente

**2. "Dérive" sans retour à la moyenne**

Contrairement à un processus stationnaire qui oscille autour d'une moyenne constante :
- Une série avec racine unitaire peut **s'éloigner indéfiniment** de son point de départ
- Elle n'a **pas de tendance à revenir** vers une valeur moyenne
- Les chocs ont un **effet permanent**

**3. Comparaison : avec vs sans racine unitaire**

| Caractéristique | $\phi < 1$ (Stationnaire) | $\phi = 1$ (Racine unitaire) |
|-----------------|---------------------------|------------------------------|
| Variance | Constante | Croissante avec t |
| Retour à la moyenne | Oui | Non |
| Effet d'un choc | Transitoire | Permanent |
| Prévisibilité | Bonne à long terme | Dégradée à long terme |
| Corrélation | Décroît exponentiellement | Persiste |

#### Exemple concret : température vs prix d'actions

**Température (stationnaire, $\phi < 1$)** :
- Oscillation autour d'une moyenne saisonnière
- Une journée anormalement chaude n'affecte pas la température dans 6 mois
- Variance stable

**Prix d'actions (racine unitaire, $\phi \approx 1$)** :
- Pas de retour à un "prix moyen"
- Une hausse de 10€ aujourd'hui affecte le prix dans 6 mois
- Variance augmente avec le temps (incertitude croissante)

### 2.4 Principe du test ADF (Augmented Dickey-Fuller)

Le test ADF est conçu pour **détecter la présence d'une racine unitaire**.

#### Formulation mathématique

Au lieu de tester directement $\phi = 1$ dans $Y_t = \phi Y_{t-1} + \epsilon_t$, le test ADF reformule le modèle :

$$\Delta Y_t = \alpha + \beta t + \gamma Y_{t-1} + \sum_{i=1}^{p} \delta_i \Delta Y_{t-i} + \epsilon_t$$

où :
- $\Delta Y_t = Y_t - Y_{t-1}$ (première différence)
- $\gamma = \phi - 1$
- $\alpha$ = constante (*drift* = dérive)
- $\beta t$ = tendance déterministe
- Les termes $\sum_{i=1}^{p} \delta_i \Delta Y_{t-i}$ capturent l'autocorrélation d'ordre supérieur

**Le test porte sur le coefficient $\gamma$** :

**Hypothèses** :
- $H_0$ : $\gamma = 0$ (équivalent à $\phi = 1$) → **racine unitaire** → série **non-stationnaire**
- $H_1$ : $\gamma < 0$ (équivalent à $\phi < 1$) → **pas de racine unitaire** → série **stationnaire**

#### Pourquoi H₀ indique la non-stationnarité ?

Si $\gamma = 0$, alors $\phi = 1$ et le modèle devient :

$$\Delta Y_t = \alpha + \beta t + \sum_{i=1}^{p} \delta_i \Delta Y_{t-i} + \epsilon_t$$

Ce qui équivaut à :

$$Y_t = Y_{t-1} + \alpha + \beta t + \sum_{i=1}^{p} \delta_i \Delta Y_{t-i} + \epsilon_t$$

C'est une **marche aléatoire avec dérive** : la série s'accumule (non-stationnaire).

Si $\gamma < 0$, alors $\phi < 1$ et le terme $\gamma Y_{t-1}$ crée une **force de rappel** vers la moyenne, rendant la série stationnaire.

#### Statistique de test

Le test calcule la **statistique t** du coefficient $\gamma$ :

$$ADF = \frac{\hat{\gamma}}{SE(\hat{\gamma})}$$

Cette statistique suit une **distribution de Dickey-Fuller** (non standard), avec des valeurs critiques tabulées.

**Interprétation** :
- Si la statistique ADF est **très négative** (< valeurs critiques) : on rejette $H_0$ → **stationnaire**
- Si la statistique ADF est **proche de 0** : on ne peut pas rejeter $H_0$ → **non-stationnaire**

#### Utilisation pratique via la p-value

En pratique, on utilise la **p-value** associée au test :

- Si **p-value < 0.05** : on rejette $H_0$ → la série est **stationnaire**
- Si **p-value ≥ 0.05** : on ne peut pas rejeter $H_0$ → la série est **non-stationnaire**

#### Pourquoi "Augmented" (Augmenté) ?

Le test original de Dickey-Fuller simple ne prend pas en compte les autocorrélations d'ordre supérieur.

Le test **ADF** ajoute les termes $\sum_{i=1}^{p} \delta_i \Delta Y_{t-i}$ pour :
- Capturer les structures AR(p) plus complexes
- S'assurer que les résidus $\epsilon_t$ sont un bruit blanc
- Rendre le test plus robuste

Le nombre de lags $p$ est généralement choisi automatiquement par le critère AIC.


#### Fonction utilitaire pour le test ADF


In [ ]:
def test_stationnarite(serie, nom='Série'):
    """
    Effectue le test ADF et affiche les résultats de manière claire.
    
    Paramètres:
    -----------
    serie : array-like
        La série temporelle à tester
    nom : str
        Nom de la série (pour l'affichage)
    
    Retour:
    -------
    dict : Résultats du test
    """
    # Supprimer les valeurs NaN
    serie_clean = serie.dropna() if isinstance(serie, pd.Series) else pd.Series(serie).dropna()
    
    # Test ADF
    resultat = adfuller(serie_clean, autolag='AIC')
    
    # Extraction des résultats
    statistic = resultat[0]
    pvalue = resultat[1]
    n_lags = resultat[2]
    n_obs = resultat[3]
    valeurs_critiques = resultat[4]
    
    # Affichage formaté
    print(f"\n{'='*60}")
    print(f"Test ADF pour : {nom}")
    print(f"{'='*60}")
    print(f"Statistique ADF : {statistic:.6f}")
    print(f"P-value : {pvalue:.6f}")
    print(f"Nombre de lags utilisés : {n_lags}")
    print(f"Nombre d'observations : {n_obs}")
    print(f"\nValeurs critiques :")
    for seuil, valeur in valeurs_critiques.items():
        print(f"  {seuil}: {valeur:.3f}")
    
    # Conclusion
    print(f"\n{'─'*60}")
    if pvalue < 0.05:
        print(f"CONCLUSION : La série '{nom}' est STATIONNAIRE")
        print(f"   (p-value = {pvalue:.6f} < 0.05 → on rejette H0)")
    else:
        print(f"CONCLUSION : La série '{nom}' est NON-STATIONNAIRE")
        print(f"   (p-value = {pvalue:.6f} ≥ 0.05 → on ne rejette pas H0)")
    print(f"{'='*60}\n")
    
    return {
        'statistic': statistic,
        'pvalue': pvalue,
        'lags': n_lags,
        'nobs': n_obs,
        'critical_values': valeurs_critiques,
        'stationnaire': pvalue < 0.05
    }

#### EXERCICE 2 : test ADF

Appliquez le test ADF aux séries générées précédemment. Les conclusions sont elles correctes ?

In [ ]:
# EXERCICE 2 : VOTRE CODE !

# Test sur le bruit blanc (devrait être stationnaire)

# Test sur la série avec tendance (devrait être non-stationnaire)

# Test sur la marche aléatoire (devrait être non-stationnaire)

# Test sur le processus AR(1) (devrait être stationnaire)

# Test sur notre série de ventes (devrait être non-stationnaire à cause de la tendance)


## 3. Différenciation et Processus Stochastiques

### 3.1 : Rendre une série stationnaire par différenciation

#### 3.1.1 Principe de la différenciation

La **différenciation** est une transformation qui permet de rendre une série non-stationnaire stationnaire en éliminant la tendance.

##### Différenciation d'ordre 1

$$\nabla Y_t = Y_t - Y_{t-1}$$

On calcule la différence entre chaque observation et la précédente.

##### Différenciation d'ordre 2

$$\nabla^2 Y_t = \nabla Y_t - \nabla Y_{t-1} = (Y_t - Y_{t-1}) - (Y_{t-1} - Y_{t-2})$$

On applique la différenciation deux fois.

#### Quand utiliser la différenciation ?

- **Tendance linéaire** → différenciation d'ordre 1 suffit généralement
- **Tendance quadratique** → différenciation d'ordre 2 peut être nécessaire
- **Marche aléatoire** → différenciation d'ordre 1 rend la série stationnaire

**Attention** : ne pas sur-différencier ! Cela peut introduire des corrélations artificielles.

#### 3.1.2 Exemple : Différenciation d'une série avec tendance

In [ ]:
# Création d'une série avec tendance forte
np.random.seed(42)
n = 200
temps = np.arange(n)

# Série originale : tendance + bruit
tendance = 0.5 * temps
bruit = np.random.normal(0, 5, n)
serie_tendance = tendance + bruit

# Conversion en pandas Series
dates = pd.date_range(start='2020-01-01', periods=n, freq='D')
serie_originale = pd.Series(serie_tendance, index=dates, name='Série originale')

print("Série avec tendance créée")
print(f"Observations : {len(serie_originale)}")

In [ ]:
# Test de stationnarité sur la série originale
stationnarite_serie_tendance_forte = test_stationnarite(serie_originale, "Série Originale (avec tendance)")

Appliquez une différenciation d’ordre 1 :

In [ ]:
# VOTRE CODE
# attention, le processus de différenciation va faire apparaître des valeurs nulles, 
# des observations seront perdues -> .dropna()



Testons la stationnarité :

In [ ]:
# VOTRE CODE 
# Test de stationnarité sur la série différenciée


In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Série originale
axes[0].plot(serie_originale, linewidth=1.5, color='red', alpha=0.7)
axes[0].set_title('Série Originale (Non-Stationnaire)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Valeur', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Série différenciée
axes[1].plot(serie_diff1, linewidth=1.5, color='green', alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].set_title('Série Différenciée d\'ordre 1 (Stationnaire)', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=12)
axes[1].set_ylabel('Différence', fontsize=12)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

#### Conclusions

- La série originale a une **tendance croissante claire** → non-stationnaire
- Après différenciation, la série **oscille autour de 0** → stationnaire
- La différenciation a **éliminé la tendance** en calculant les changements successifs

#### Exercice 3 : Marche aléatoire et différenciation

Voici un bout de code qui crée une série en marche aléatoire. Comme précédemment, rendez cette série stationnaire à l’aide de la technique de la différentiation.

In [ ]:
# Création d'une marche aléatoire
np.random.seed(123)
n = 300
innovations = np.random.normal(0, 1, n)
marche_aleatoire = np.cumsum(innovations)  # Somme cumulative

dates_rw = pd.date_range(start='2020-01-01', periods=n, freq='D')
serie_rw = pd.Series(marche_aleatoire, index=dates_rw, name='Marche aléatoire')

print("Marche aléatoire : Y_t = Y_{t-1} + ε_t")
print(f"où ε_t ~ N(0, 1)")

In [ ]:
# EXERCICE 3 : VOTRE CODE !



In [ ]:
# Visualisation
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Marche aléatoire
axes[0].plot(serie_rw, linewidth=1.5, color='navy', alpha=0.7)
axes[0].set_title('Marche Aléatoire : Y_t = Y_{t-1} + ε_t', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Y_t', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Série différenciée (= innovations originales)
axes[1].plot(serie_rw_diff, linewidth=1, color='green', alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].set_title('Différence : ∇Y_t = Y_t - Y_{t-1} = ε_t (Bruit Blanc)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('∇Y_t', fontsize=12)
axes[1].grid(True, alpha=0.3)

# Innovations originales (pour comparaison)
axes[2].plot(dates_rw, innovations, linewidth=1, color='orange', alpha=0.7)
axes[2].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[2].set_title('Innovations Originales ε_t (pour vérification)', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Date', fontsize=12)
axes[2].set_ylabel('ε_t', fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

##### Conclusion

À quoi est égal le bruit blanc obtenu par différentiation ?


In [ ]:
# Votre réponse

#### Exercice 4 : Différenciation d'une série de prix

Les prix d'actions suivent souvent une marche aléatoire, mais les rendements (variations relatives) sont généralement stationnaires. Modélisons le rendement et le prix d’une action imaginaire, puis testons la stationnarité.

1. Créez une série de prix d'actions simulée (marche aléatoire avec dérive positive)
2. Testez la stationnarité de la série de prix
3. Calculez les rendements (différence logarithmique ou différence simple). Affichez les statistique courante des rendements générés (moyenne, dispersion, etc.)
    Rappels :
    - Rendements simples : $R_t= \frac{P_t-P_{t−1}}{Pt-1}=\frac{Pt}{P_{t−1}}−1$
    - Taux de croissance continue (log-returns) : $r_t = \ln \frac{P_{t−1}}{P_t} = \ln P_t − \ln P_{t−1}$
5. Testez la stationnarité des rendements
6. Après visualisation des deux séries, quelles sont vos conclusions ?

**Note :** pandas et numpy proposent des méthodes qui permettent d’écrire simplement les calculs pour des séries : `.diff()`, `.shift()`, `.pct_change()`. N’oubliez pas de supprimer les valeurs manquantes (`.dropna()`) en début ou en fin de série qui ne manquerons pas d’apparaître quand on réalise des opérations entre un élément et le précédent/suivant d’une série.

In [ ]:
# EXERCICE 4 - VOTRE CODE 


In [ ]:
# 2. Test de stationnarité sur les prix
# VOTRE CODE


In [ ]:
# 3. Calculer les rendements (diff simple ou log)



In [ ]:
# 4. Test de stationnarité sur les rendements
# VOTRE CODE



In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(3, 1, figsize=(14, 12))

# Prix
axes[0].plot(serie_prix, linewidth=1.5, color='darkblue')
axes[0].set_title('Prix de l\'Action (Non-Stationnaire)', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Prix (€)', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Rendements logarithmiques
axes[1].plot(rendements_log, linewidth=1, color='green', alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].set_title('Rendements Logarithmiques (Stationnaire)', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Log-return', fontsize=12)
axes[1].grid(True, alpha=0.3)

# Distribution des rendements
axes[2].hist(rendements_log, bins=50, color='green', alpha=0.7, edgecolor='black')
axes[2].axvline(x=rendements_log.mean(), color='red', linestyle='--', linewidth=2, label=f'Moyenne = {rendements_log.mean():.4f}')
axes[2].set_title('Distribution des Rendements', fontsize=14, fontweight='bold')
axes[2].set_xlabel('Log-return', fontsize=12)
axes[2].set_ylabel('Fréquence', fontsize=12)
axes[2].legend(fontsize=11)
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

Vos conclusions ?

In [ ]:
# conclusion = 

### 3.2 : Résumé des processus Stochastiques de Base

#### 3.2.1 Bruit Blanc (White Noise)

Le **bruit blanc** est le processus stochastique le plus simple.

##### Définition

Un processus $\{\epsilon_t\}$ est un bruit blanc si :

1. $E[\epsilon_t] = 0$ pour tout $t$ (moyenne nulle)
2. $Var(\epsilon_t) = \sigma^2$ pour tout $t$ (variance constante)
3. $Cov(\epsilon_t, \epsilon_s) = 0$ pour tout $t \neq s$ (pas de corrélation)

Notation : $\epsilon_t \sim WN(0, \sigma^2)$

##### Propriétés

- Le bruit blanc est **stationnaire**
- Il est **imprévisible** : la meilleure prévision est la moyenne (0)
- C'est le "building block" des modèles de séries temporelles

##### Exemple : générer un bruit blanc

In [ ]:
# Génération d'un bruit blanc
np.random.seed(42)
n = 500
bruit_blanc = np.random.normal(0, 1, n)

# Visualisation
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Série temporelle
axes[0].plot(bruit_blanc, linewidth=1, color='gray', alpha=0.7)
axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[0].set_title('Bruit Blanc : ε_t ~ N(0, 1)', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Temps', fontsize=12)
axes[0].set_ylabel('Valeur', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Histogramme
axes[1].hist(bruit_blanc, bins=40, color='gray', alpha=0.7, edgecolor='black', density=True)
# Courbe normale théorique
x = np.linspace(-4, 4, 100)
axes[1].plot(x, (1/np.sqrt(2*np.pi)) * np.exp(-x**2/2), 'r-', linewidth=2, label='N(0,1) théorique')
axes[1].set_title('Distribution du Bruit Blanc', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Valeur', fontsize=12)
axes[1].set_ylabel('Densité', fontsize=12)
axes[1].legend(fontsize=11)
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print(f"Moyenne empirique : {bruit_blanc.mean():.4f} (théorique : 0)")
print(f"Écart-type empirique : {bruit_blanc.std():.4f} (théorique : 1)")

#### 3.2.2 Processus AutoRégressif (AR)

Un processus **autorégressif d'ordre p**, noté AR(p), utilise les p valeurs passées pour prédire la valeur actuelle.

##### AR(1) - AutoRégressif d'ordre 1

$$Y_t = c + \phi Y_{t-1} + \epsilon_t$$

où :
- $c$ est une constante
- $\phi$ est le coefficient autorégressif
- $\epsilon_t \sim WN(0, \sigma^2)$ est le bruit blanc

##### Condition de stationnarité

Un processus AR(1) est **stationnaire si et seulement si** : $|\phi| < 1$

- Si $\phi = 0$ : on retrouve un bruit blanc
- Si $\phi = 1$ : on a une marche aléatoire (non-stationnaire)
- Si $\phi > 0$ : corrélation positive (persistance)
- Si $\phi < 0$ : corrélation négative (oscillations)

##### AR(p) - Forme générale

$$
AR(p): Y_t = c + \phi_1 Y_{t-1} + \phi_2 Y_{t-2}  + … + \phi_p Y_{t-p} + \varepsilon_t
$$

##### AR(p) - Forme avec facteur de retard 

$$
AR(p): Y_t = c + \phi_1 Y_{t-1} + \phi_2 Y_{t-2}  + … + \phi_p Y_{t-p} + \varepsilon_t \\
(1- \varphi_1 L-\varphi_2 L^2-\ldots-\varphi_p L^p) Y_t = c + \varepsilon_t 
$$

En définissant l’opérateur de retard  ainsi: $L Y_t = Y_{t-1}$

In [ ]:
# Fonction pour simuler un processus AR(1)
def simuler_ar1(phi, c=0, n=500, sigma=1, y0=0):
    """
    Simule un processus AR(1) : Y_t = c + phi * Y_{t-1} + epsilon_t
    
    Paramètres:
    -----------
    phi : float
        Coefficient autorégressif
    c : float
        Constante
    n : int
        Nombre d'observations
    sigma : float
        Écart-type du bruit blanc
    y0 : float
        Valeur initiale
    """
    y = [y0]
    epsilon = np.random.normal(0, sigma, n)
    
    for t in range(n-1):
        y_next = c + phi * y[-1] + epsilon[t]
        y.append(y_next)
    
    return np.array(y)

Processus $AR(1)$ simulés avec différents coefficients $\phi$

In [ ]:
# Simulation de différents AR(1)
np.random.seed(42)
n = 300

ar1_phi05 = simuler_ar1(phi=0.5, n=n)    # Stationnaire, corrélation positive
ar1_phi09 = simuler_ar1(phi=0.9, n=n)    # Stationnaire, forte persistance
ar1_phi_neg = simuler_ar1(phi=-0.7, n=n) # Stationnaire, oscillations
ar1_phi1 = simuler_ar1(phi=1.0, n=n)     # Non-stationnaire (marche aléatoire)

# Visualisation comparative
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Processus AR(1) avec Différents Coefficients', fontsize=16, fontweight='bold', y=0.995)

# φ = 0.5 (stationnaire)
axes[0, 0].plot(ar1_phi05, linewidth=1, color='green')
axes[0, 0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[0, 0].set_title('AR(1) avec φ = 0.5 (Stationnaire)', fontsize=13, fontweight='bold')
axes[0, 0].set_ylabel('Y_t', fontsize=11)
axes[0, 0].grid(True, alpha=0.3)

# φ = 0.9 (stationnaire mais très persistant)
axes[0, 1].plot(ar1_phi09, linewidth=1, color='orange')
axes[0, 1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[0, 1].set_title('AR(1) avec φ = 0.9 (Forte Persistance)', fontsize=13, fontweight='bold')
axes[0, 1].set_ylabel('Y_t', fontsize=11)
axes[0, 1].grid(True, alpha=0.3)

# φ = -0.7 (oscillations)
axes[1, 0].plot(ar1_phi_neg, linewidth=1, color='purple')
axes[1, 0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[1, 0].set_title('AR(1) avec φ = -0.7 (Oscillations)', fontsize=13, fontweight='bold')
axes[1, 0].set_xlabel('Temps', fontsize=11)
axes[1, 0].set_ylabel('Y_t', fontsize=11)
axes[1, 0].grid(True, alpha=0.3)

# φ = 1.0 (marche aléatoire - non-stationnaire)
axes[1, 1].plot(ar1_phi1, linewidth=1, color='red')
axes[1, 1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[1, 1].set_title('AR(1) avec φ = 1.0 (Marche Aléatoire)', fontsize=13, fontweight='bold')
axes[1, 1].set_xlabel('Temps', fontsize=11)
axes[1, 1].set_ylabel('Y_t', fontsize=11)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Visualisation d’un processus explosif :

In [ ]:
ar1_phi12 = simuler_ar1(phi=1.2, n=50) # Non-stationnaire, explosivité

fig, axes = plt.subplots(figsize=(16, 10))
fig.suptitle('Processus AR(1) explosif', fontsize=16, fontweight='bold', y=0.995)

# φ = 1.2 (explosif)
axes.plot(ar1_phi12, linewidth=1, color='green')
axes.set_ylabel('Y_t', fontsize=11)
axes.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Comment interprêtez-vous les différentes valeurs de $\phi$ sur ces visualisations ?

In [ ]:
conclusion = '''
- φ = 0.5 : la valeur actuelle dépend modérément de la valeur précédente
- φ = 0.9 : forte dépendance → la série a une "mémoire longue"
- φ = -0.7 : alternance entre valeurs positives et négatives
- φ = 1.0 : marche aléatoire → non-stationnaire (racine unitaire)
- φ = 1.2 : explosivité → l’influence passée est exponentielle plus le temps passe'''

print(conclusion)

#### 3.2.3 Processus Moyenne Mobile (MA)

Un processus **moyenne mobile d'ordre q**, noté MA(q), utilise les q erreurs passées.

##### MA(1) - Moyenne Mobile d'ordre 1

$$Y_t = \mu + \epsilon_t + \theta \epsilon_{t-1}$$

où :
- $\mu$ est la moyenne
- $\theta$ est le coefficient de moyenne mobile
- $\epsilon_t \sim WN(0, \sigma^2)$ est le bruit blanc

##### Propriétés

- Un processus MA(q) est **toujours stationnaire** (peu importe $\theta$)
- La corrélation est nulle au-delà du lag q ("mémoire courte")
- Contrairement aux AR, les MA sont toujours inversibles

##### MA(q) - Forme générale

$$Y_t = \mu + \epsilon_t + \theta_1 \epsilon_{t-1} + \theta_2 \epsilon_{t-2} + ... + \theta_q \epsilon_{t-q}$$

##### Interprétation physique fondamentale

Un processus MA modélise un système où les chocs (perturbations, innovations) ont un effet qui persiste pendant un temps limité. Après q périodes, l'effet du choc disparaît complètement. C'est une "mémoire courte" : le système "oublie" rapidement. On peut faire une analogie simple sur ce qu’il se passe quand on jette des cailloux dans un étang, on peut imaginer que les ondes formées dans l’étant obéissent à un processus MA(2) :
- t=0 : Vous lancez un caillou (choc ε₀)
      → Onde à la surface

- t=1 : L'onde se propage (effet de ε₀)
      + vous lancez un nouveau caillou (choc ε₁)
      → Superposition de deux ondes

- t=2 : Les deux ondes se propagent encore
      + nouveau caillou (choc ε₂)
      → 3 ondes actives

- t=3 : L'onde de t=0 a disparu (vu que c’est un processus $MA(2)$ mémoire = 2 périodes)
      Seules les ondes de t=1 et t=2 restent
      + nouveau caillou

  
Ce que vous observez (niveau de l'eau = $Y_t$) est la superposition des effets des 2 derniers cailloux seulement.

##### Exemple réels :

-  Gestion des stocks d'un supermarché (inventaire avec délai de livraison)
   Le niveau de stock observé aujourd'hui dépend de :
    - Les ventes/livraisons aléatoires du jour J
    - Les commandes passées à J-1 (délai de livraison)
    - Les commandes passées il y a 2 jours (J-2)

    $Stock_t = Stock_moyen + choc_t + effet_commande_{t-1} + effet_commande_{t-2}$

    Si le délai de livraison maximum est de 2 jours, le système a une mémoire de 2 : on a un processus MA(2)

- Modèle météo qui s'auto-corrige (correction d’erreur/de mesure)
    Un centre météo fait une prévision, puis observe l'erreur et corrige :
    - Jour J : prévision avec erreur ε_t
    - J+1 : on corrige partiellement l'erreur de J (θ₁ ε_t)
    - J+2 : on corrige l'erreur de J+1 (θ₂ ε_{t-1})

    La température prédite reflète $Temp_t = Tendance + ε_t + θ₁ ε_{t-1} + θ₂ ε_{t-2}$

- Concentration d'un médicament dans le sang (pharmacodynamique)

    Un médicament pris chaque jour a une demi-vie de 2 jours :
    - Jour 0 : vous prenez 100mg (choc ε₀)
    - Jour 1 : il reste 50mg de hier + 100mg aujourd'hui
    - Jour 2 : il reste 25mg de J0 + 50mg de J1 + 100mg de J2
    - Jour 3 : J0 est éliminé (< 12.5mg négligeable)
    $Concentration_t ≈ ε_t + 0.5 ε_{t-1} + 0.25 ε_{t-2}$

##### Importance des processus MA
Ces processus sont impliquée dans la modélisation de nombreuses situations :

1. Systèmes avec retards/délais
    Beaucoup de systèmes réels ont des délais :
   * Production → vente (délai logistique)
   * Investissement → rendement (délai de construction)
   * Politique économique → effet (délai de transmission)
   Ces délais créent naturellement des structures MA.
2. Agrégation de processus
    Même si les processus élémentaires sont AR, leur agrégation peut créer un MA. Exemple :
    * 100 magasins avec leurs propres dynamiques AR
    * L'agrégation (ventes totales) peut ressembler à un MA !

3. Erreurs de mesure dans les données
    En pratique, toute série observée contient des erreurs :
    * $Y_{observé} = Y_{réel} + erreur_{mesure}$
    * Si les erreurs sont corrélées sur quelques périodes → composante MA !

##### Processus MA vs. AR

Les processus AR sont impactés par les niveaux passés (une inertie), un processus MA par les chocs récents (qui vont finir par s’évanouir).

* Processus AR (AutoRégressif)
    * Mécanisme : Le système lui-même a de l'inertie. C'est le niveau qui persiste
    * Exemple : température (inertie thermique)
    * formule : $Y_t = φ Y_{t-1} + ε_t$
    * Analogie : Un volant d'inertie qui tourne. Si vous le poussez (choc), il continue à tourner (persistance). Le mouvement actuel dépend du mouvement passé.

* Processus MA (Moyenne Mobile)
    * Mécanisme : Les perturbations ont un effet qui s'estompe. C'est le choc qui persiste temporairement
    * Exemple : onde à la surface de l'eau
    * formule : Y_t = ε_t + θ ε_{t-1}
    * Analogie : Caillou dans l'étang. L'onde (effet du choc) se propage puis disparaît. Le niveau actuel dépend des chocs récents, pas du niveau passé.
* 
* | Type | Processus AR (stationnaire) | Processus MA |
|------|-----------------------------|--------------|
| Inertie | Le système lui-même | Les perturbations |
| Persistance | Décroît exp. / infinie | Finie (q périodes) |
| Mémoire | Longue | Courte |
| Exemples | Température, PIB, inflation, prix actions | Erreurs de mesure, stocks avec délais, corrections successives |
| Retour à la moyenne | Graduel | Rapide (après q périodes) |
| Analogie physique | Système avec masse/inertie | Perturbation impulsionnelle |

##### Exemple (simulation/visualisation)

In [ ]:
# Fonction pour simuler un processus MA(1)
def simuler_ma1(theta, mu=0, n=500, sigma=1):
    """
    Simule un processus MA(1) : Y_t = mu + epsilon_t + theta * epsilon_{t-1}
    """
    epsilon = np.random.normal(0, sigma, n+1)
    y = np.zeros(n)
    
    for t in range(n):
        y[t] = mu + epsilon[t+1] + theta * epsilon[t]
    
    return y

# Simulation de différents MA(1)
np.random.seed(42)
n = 300

ma1_theta05 = simuler_ma1(theta=0.5, n=n)
ma1_theta09 = simuler_ma1(theta=0.9, n=n)
ma1_theta_neg = simuler_ma1(theta=-0.7, n=n)

print("Processus MA(1) simulés")

In [ ]:
# Visualisation
fig, axes = plt.subplots(3, 1, figsize=(14, 12))
fig.suptitle('Processus MA(1) avec Différents Coefficients', fontsize=16, fontweight='bold', y=0.995)

# θ = 0.5
axes[0].plot(ma1_theta05, linewidth=1, color='steelblue', alpha=0.7)
axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[0].set_title('MA(1) avec θ = 0.5', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Y_t', fontsize=11)
axes[0].grid(True, alpha=0.3)

# θ = 0.9
axes[1].plot(ma1_theta09, linewidth=1, color='forestgreen', alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[1].set_title('MA(1) avec θ = 0.9', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Y_t', fontsize=11)
axes[1].grid(True, alpha=0.3)

# θ = -0.7
axes[2].plot(ma1_theta_neg, linewidth=1, color='crimson', alpha=0.7)
axes[2].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[2].set_title('MA(1) avec θ = -0.7', fontsize=13, fontweight='bold')
axes[2].set_xlabel('Temps', fontsize=11)
axes[2].set_ylabel('Y_t', fontsize=11)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Les processus MA ont l'air plus « bruités » que les processus AR car ils n'ont pas de composante autorégressive (pas de mémoire longue)

Contrairement à AR(1) où φ contrôle la persistance à long terme, θ dans MA(1) ne contrôle que la corrélation sur 1 seule période, puis tout s'arrête ! C'est une "mémoire flash" de 1 période. 

Nous ne faisons pas la démonstration dans ce cours (qui demande de calculer le lien entre θ et la variance), mais retenez :

* θ ≈ 0 → Bruit blanc, erratique
* θ > 0 élevé → Courbe lisse, vagues douces
* θ < 0 élevé → Zigzag, oscillations rapides

#### 3.2.4 Modèles hybrides ARMA

Un processus **ARMA(p,q)** combine les composantes AR et MA.

En effet, dans la réalité, la plupart des phénomènes ont à la fois :

* de l'inertie (composante AR)
* des chocs qui persistent temporairement (composante MA)

##### Exemple : les cours boursiers

Les rendements boursiers peuvent être modélisés comme un processus hybride ARMA(1,1) :

$$r_t = \phi r_{t-1} + \varepsilon_t + \theta \varepsilon_{t-1}$$

* $\phi r_{t-1}$ -> AR (momentum)
* $\varepsilon_t + \theta \varepsilon_{t-1}$ -> AM (sur-réaction puis correction du marché)

* AR : le momentum, soit une tendance continue sur quelques temps -> quand une valeur monte ou descends, les autres investisseurs « suivent le mouvement » et achètent pour profiter de la montée ou vendent pour limiter les pertent et amplifient le mouvement : c’est le *momentum* ou inertie.
* MA : sur-réaction puis correction sur un temps plus court (le marché surréagit aux nouvelles puis se corrige ou « rebondit » au bout de quelques jours ou dès le lendemain)

##### ARMA(p,q) - Forme générale

$$Y_t = c + \phi_1 Y_{t-1} + ... + \phi_p Y_{t-p} + \epsilon_t + \theta_1 \epsilon_{t-1} + ... + \theta_q \epsilon_{t-q}$$

##### Exemple : ARMA(1,1)

$$Y_t = c + \phi Y_{t-1} + \epsilon_t + \theta \epsilon_{t-1}$$

##### Avantages

- Plus **parcimonieux** : ARMA(1,1) peut modéliser ce qu'un AR(∞) modéliserait
- Combine la **mémoire longue** (AR) et les **chocs transitoires** (MA)
- Souvent meilleur ajustement avec moins de paramètres

##### Exemple : simulation d’un processus ARMA(1,1)

In [ ]:
# Simulation d'un ARMA(1,1)
def simuler_arma11(phi, theta, c=0, n=500, sigma=1, y0=0):
    """
    Simule un processus ARMA(1,1) : Y_t = c + phi*Y_{t-1} + epsilon_t + theta*epsilon_{t-1}
    """
    y = [y0]
    epsilon = np.random.normal(0, sigma, n+1)
    
    for t in range(1, n):
        y_next = c + phi * y[-1] + epsilon[t] + theta * epsilon[t-1]
        y.append(y_next)
    
    return np.array(y)

np.random.seed(42)
arma11 = simuler_arma11(phi=0.7, theta=0.4, n=500)

# Visualisation
plt.figure(figsize=(14, 6))
plt.plot(arma11, linewidth=1, color='darkviolet', alpha=0.7)
plt.axhline(y=0, color='black', linestyle='--', alpha=0.3)
plt.title('Processus ARMA(1,1) avec φ=0.7 et θ=0.4', fontsize=14, fontweight='bold')
plt.xlabel('Temps', fontsize=12)
plt.ylabel('Y_t', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("Processus ARMA(1,1) simulé")
print(f"Moyenne : {arma11.mean():.4f}")
print(f"Écart-type : {arma11.std():.4f}")

N’hésitez pas à modifier les valeurs de $\phi$ et $\theta$ pour bien comprendre commet ces paramètres affectent la forme de la courbe :
- $\phi = 1$
- $\phi > 1$
- $\theta < 0$
- $\theta = 1$
- etc.

##### Exercice : comparer AR, MA et ARMA

Comprendre visuellement les différences entre AR, MA et ARMA

1. Simulez les processus suivants (n=500) :
   - AR(2) : $Y_t = 0.6·Y_{t-1} - 0.3·Y_{t-2} + ε_t$
   - MA(2) : $Y_t = ε_t + 0.7·ε_{t-1} + 0.8·ε_{t-2}$
   - ARMA(1,1) : $Y_t = 0.8·Y_{t-1} + ε_t + 0.4·ε_{t-1}$
2. Visualisez les trois séries
3. Testez leur stationnarité
4. Identifiez visuellement les différences

In [ ]:
# EXERCICE 3 - VOTRE CODE

# 1. Les trois processus
# ar2 = ...
# ma2 = ...
# arma11 = ...

# 2. Visualisation et tests
# ...

In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(3, 1, figsize=(14, 12))
fig.suptitle('Comparaison : AR(2) vs MA(2) vs ARMA(1,1)', fontsize=16, fontweight='bold', y=0.995)

# AR(2)
axes[0].plot(ar2, linewidth=1, color='royalblue', alpha=0.7)
axes[0].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[0].set_title('AR(2) : Y_t = 0.6·Y_{t-1} - 0.3·Y_{t-2} + ε_t', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Y_t', fontsize=11)
axes[0].grid(True, alpha=0.3)

# MA(2)
axes[1].plot(ma2, linewidth=1, color='forestgreen', alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[1].set_title('MA(2) : Y_t = ε_t + 0.7·ε_{t-1} + 0.8·ε_{t-2}', fontsize=13, fontweight='bold')
axes[1].set_ylabel('Y_t', fontsize=11)
axes[1].grid(True, alpha=0.3)

# ARMA(1,1)
axes[2].plot(arma11_ex, linewidth=1, color='darkorange', alpha=0.7)
axes[2].axhline(y=0, color='black', linestyle='--', alpha=0.3)
axes[2].set_title('ARMA(1,1) : Y_t = 0.8·Y_{t-1} + ε_t + 0.4·ε_{t-1}', fontsize=13, fontweight='bold')
axes[2].set_xlabel('Temps', fontsize=11)
axes[2].set_ylabel('Y_t', fontsize=11)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Tests de stationnarité ar2


In [ ]:
# Tests de stationnarité ma2


In [ ]:
# Tests de stationnarité arma11


In [ ]:
# Statistiques comparatives
print("\n" + "="*70)
print("STATISTIQUES COMPARATIVES")
print("="*70)

stats_df = pd.DataFrame({
    'Processus': ['AR(2)', 'MA(2)', 'ARMA(1,1)'],
    'Moyenne': [ar2.mean(), ma2.mean(), arma11_ex.mean()],
    'Écart-type': [ar2.std(), ma2.std(), arma11_ex.std()],
    'Min': [ar2.min(), ma2.min(), arma11_ex.min()],
    'Max': [ar2.max(), ma2.max(), arma11_ex.max()]
})

print(stats_df.to_string(index=False))

print("\nObservations :")
print("  - AR(2) : oscillations plus régulières/lisses et stables autour de l’axe, (mémoire autorégressive)")
print("  - MA(2) : variations plus erratiques, avec des soubressauts/tendances (mémoire courte)")
print("  - ARMA(1,1) : comportement intermédiaire")
print("  - Tous sont stationnaires !")

## 4. ACF/PACF et Modèles ARIMA

### 4.1 Autocorrélation et caractérisation des processus

#### 4.1.1 Fonction d'Autocorrélation (ACF)

##### Définition

L'**autocorrélation** mesure la corrélation entre une série et ses valeurs avec un certain décalage (lags).

Pour un lag $k$, l'autocorrélation est :

$$\rho_k = \frac{Cov(Y_t, Y_{t-k})}{\sqrt{Var(Y_t) \cdot Var(Y_{t-k})}} = \frac{Cov(Y_t, Y_{t-k})}{Var(Y_t)}$$

Pour une série stationnaire : $\rho_k = \frac{\gamma_k}{\gamma_0}$

##### Interprétation

- $\rho_k$ proche de 1 : forte corrélation positive au lag $k$
- $\rho_k$ proche de -1 : forte corrélation négative au lag $k$
- $\rho_k$ proche de 0 : pas de corrélation au lag $k$

##### Patterns typiques de l'ACF

- **Bruit blanc** : tous les $\rho_k \approx 0$ pour $k > 0$
- **AR(p)** : décroissance exponentielle ou sinusoïdale
- **MA(q)** : coupure nette après le lag $q$ (tous les $\rho_k = 0$ pour $k > q$)
- **Tendance** : décroissance très lente vers 0

#### 4.1.2 Fonction d'Autocorrélation Partielle (PACF)

##### Définition

La **PACF** mesure la corrélation entre $Y_t$ et $Y_{t-k}$ **après avoir enlevé l'effet des lags intermédiaires** (1, 2, ..., k-1).

C'est la corrélation "directe" au lag $k$, sans l'influence des lags précédents.

##### Interprétation

La PACF aide à identifier l'ordre $p$ d'un processus AR.

##### Patterns typiques de la PACF

- **Bruit blanc** : tous les $\phi_{kk} \approx 0$
- **AR(p)** : coupure nette après le lag $p$ (tous les $\phi_{kk} = 0$ pour $k > p$)
- **MA(q)** : décroissance exponentielle ou sinusoïdale

##### Tableau récapitulatif ACF/PACF

| Processus | ACF | PACF |
|-----------|-----|------|
| **AR(p)** | Décroissance exponentielle | Coupure nette après lag p |
| **MA(q)** | Coupure nette après lag q | Décroissance exponentielle |
| **ARMA(p,q)** | Décroissance exponentielle | Décroissance exponentielle |

#### 4.1.3 Exemples : ACF et PACF de différents processus

Simulons différents processus AR(1), AM(1) et ARMA(1,1) avec les fonctions que nous avons créé dans la partie précédente :

In [ ]:
# Simulation des processus
np.random.seed(42)
n = 500

bruit_blanc = np.random.normal(0, 1, n)
ar1 = simuler_ar1(phi=0.7, n=n)
ma1 = simuler_ma1(theta=0.7, n=n)
arma11 = simuler_arma11(phi=0.6, theta=0.4, n=n)

##### Exemple 1 : ACF et PACF du bruit blanc

Pour une représentation graphique de ces analyses, utilisons les méthodes `plot_acf` et `plot_pacf` du module`statsmodels.graphics.tsaplots` pour tracer les autocorrélations et les autocorrélations partielles : 

In [ ]:
# ACF et PACF du bruit blanc
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('Bruit Blanc : ACF et PACF', fontsize=16, fontweight='bold')

# ACF
plot_acf(bruit_blanc, lags=40, ax=axes[0], alpha=0.05)
axes[0].set_title('ACF - Bruit Blanc', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Lag', fontsize=11)
axes[0].set_ylabel('Autocorrélation', fontsize=11)

# PACF
plot_pacf(bruit_blanc, lags=40, ax=axes[1], alpha=0.05, method='ywm')
axes[1].set_title('PACF - Bruit Blanc', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Lag', fontsize=11)
axes[1].set_ylabel('Autocorrélation Partielle', fontsize=11)

plt.tight_layout()
plt.show()

- Toutes les valeurs sont dans l'intervalle de confiance (zone bleue)
- Donc aucune corrélation n’est significative → confirme que c'est du bruit blanc

##### Exemple 2 : Processus AR(1)

In [ ]:
# ACF et PACF d'un AR(1)
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('AR(1) avec φ=0.7 : ACF et PACF', fontsize=16, fontweight='bold')

# ACF
plot_acf(ar1, lags=40, ax=axes[0], alpha=0.05)
axes[0].set_title('ACF - AR(1)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Lag', fontsize=11)
axes[0].set_ylabel('Autocorrélation', fontsize=11)

# PACF
plot_pacf(ar1, lags=40, ax=axes[1], alpha=0.05, method='ywm')
axes[1].set_title('PACF - AR(1)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Lag', fontsize=11)
axes[1].set_ylabel('Autocorrélation Partielle', fontsize=11)

plt.tight_layout()
plt.show()

Observations caractéristiques d'un AR(1) :
- ACF : décroissance exponentielle (lente)
- PACF : pic significatif au lag 1, puis valeurs non significatives
- on conclut que PACF coupe après le lag p=1 → indice que nous avons affaire à un processus AR(1)

##### Exemple 3 : Processus MA(1)

In [ ]:
# ACF et PACF d'un MA(1)
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('MA(1) avec θ=0.7 : ACF et PACF', fontsize=16, fontweight='bold')

# ACF
plot_acf(ma1, lags=40, ax=axes[0], alpha=0.05)
axes[0].set_title('ACF - MA(1)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Lag', fontsize=11)
axes[0].set_ylabel('Autocorrélation', fontsize=11)

# PACF
plot_pacf(ma1, lags=40, ax=axes[1], alpha=0.05, method='ywm')
axes[1].set_title('PACF - MA(1)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Lag', fontsize=11)
axes[1].set_ylabel('Autocorrélation Partielle', fontsize=11)

plt.tight_layout()
plt.show()

Observations caractéristiques d'un MA(1) :
- ACF : pic significatif au lag 1, puis valeurs non significatives
- PACF : décroissance exponentielle
- On conclut que ACF coupe après le lag q=1 → indice que nous avons affaire à un processus MA(1)

##### Exemple 4 : Processus ARMA(1,1)

In [ ]:
# ACF et PACF d'un ARMA(1,1)
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('ARMA(1,1) avec φ=0.6 et θ=0.4 : ACF et PACF', fontsize=16, fontweight='bold')

# ACF
plot_acf(arma11, lags=40, ax=axes[0], alpha=0.05)
axes[0].set_title('ACF - ARMA(1,1)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Lag', fontsize=11)
axes[0].set_ylabel('Autocorrélation', fontsize=11)

# PACF
plot_pacf(arma11, lags=40, ax=axes[1], alpha=0.05, method='ywm')
axes[1].set_title('PACF - ARMA(1,1)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Lag', fontsize=11)
axes[1].set_ylabel('Autocorrélation Partielle', fontsize=11)

plt.tight_layout()
plt.show()

Observations caractéristiques d'un ARMA(1,1) :
- ACF : décroissance exponentielle (pas de coupure nette)
- PACF : décroissance exponentielle (pas de coupure nette)
- on conclut que les deux graphiques montrent une décroissance → Combinaison AR + MA

#### 4.1.4 EXERCICE 5 : Identification de processus avec ACF/PACF

Identifier l'ordre d'un processus inconnu en analysant ACF et PACF

1. On va générer 3 processus mystères (les paramètres vont être définis aléatoirement)
2. Analysez leurs ACF et PACF
3. Identifiez le type de processus (AR, MA, ARMA) et leur ordre
4. Justifiez votre réponse

In [ ]:
# Génération des processus mystères
np.random.seed(123)
n = 500

# Mystère 1 : AR(2)
mystere1 = [0, 0]
eps1 = np.random.normal(0, 1, n)
for t in range(2, n):
    y_next = 0.5 * mystere1[-1] + 0.3 * mystere1[-2] + eps1[t]
    mystere1.append(y_next)
mystere1 = np.array(mystere1)

# Mystère 2 : MA(2)
eps2 = np.random.normal(0, 1, n+2)
mystere2 = np.zeros(n)
for t in range(n):
    mystere2[t] = eps2[t+2] + 0.6 * eps2[t+1] + 0.3 * eps2[t]

# Mystère 3 : AR(1)
mystere3 = [0]
eps3 = np.random.normal(0, 1, n)
for t in range(n-1):
    mystere3.append(0.8 * mystere3[-1] + eps3[t])
mystere3 = np.array(mystere3)

print("3 processus mystères générés")
print("Analysez leurs ACF et PACF pour les identifier !")

In [ ]:
# EXERCICE 4 - VOTRE CODE
# Tracez les ACF et PACF des 3 processus mystères
# Identifiez leur type et ordre

# Mystère 1
# fig, axes = plt.subplots(1, 2, figsize=(16, 5))
# plot_acf(mystere1, lags=20, ax=axes[0])
# plot_pacf(mystere1, lags=20, ax=axes[1], method='ywm')
# ...

# Votre analyse :
# Mystère 1 : ...
# Mystère 2 : ...
# Mystère 3 : ...

### 4.2 : Modèles ARIMA

#### 4.2.1 Qu'est-ce qu'un modèle ARIMA ?

**ARIMA** = **A**uto**R**egressive **I**ntegrated **M**oving **A**verage

##### Structure ARIMA(p, d, q)

Un modèle ARIMA combine :

1. **AR(p)** : Partie AutoRégressive d'ordre p
2. **I(d)** : Intégration (différenciation) d'ordre d
3. **MA(q)** : Partie Moyenne Mobile d'ordre q

##### Les 3 paramètres

- **p** : ordre autorégressif (nombre de lags de Y)
- **d** : ordre de différenciation (nombre de fois qu'on différencie)
- **q** : ordre moyenne mobile (nombre de lags des erreurs)

##### Équation ARIMA(p,d,q)

On applique d différenciations à la série, puis :

$$\nabla^d Y_t = c + \phi_1 \nabla^d Y_{t-1} + ... + \phi_p \nabla^d Y_{t-p} + \epsilon_t + \theta_1 \epsilon_{t-1} + ... + \theta_q \epsilon_{t-q}$$

##### Cas particuliers

- **ARIMA(p, 0, 0)** = AR(p)
- **ARIMA(0, 0, q)** = MA(q)
- **ARIMA(p, 0, q)** = ARMA(p, q)
- **ARIMA(0, 1, 0)** = Marche aléatoire
- **ARIMA(0, 1, 1)** = Lissage exponentiel simple

Tout réside donc dans la détermination des 3 paramètres p, d et q.

#### 4.2.2 Méthodologie d'identification des paramètres (p, d, q)

##### Étape 1 : Déterminer d (ordre de différenciation)

1. Tester la stationnarité de la série originale (test ADF)
2. Si non-stationnaire : appliquer différenciation d'ordre 1
3. Tester à nouveau la stationnarité
4. Répéter si nécessaire (rarement d > 2)

**Règle** : d = nombre de différenciations nécessaires pour obtenir la stationnarité

##### Étape 2 : Déterminer p et q

Sur la série **différenciée** (stationnaire), analyser ACF et PACF :

| Pattern | ACF | PACF | Modèle |
|---------|-----|------|--------|
| **AR(p)** | Décroissance exp. | Coupure après lag p | ARIMA(p, d, 0) |
| **MA(q)** | Coupure après lag q | Décroissance exp. | ARIMA(0, d, q) |
| **ARMA(p,q)** | Décroissance exp. | Décroissance exp. | ARIMA(p, d, q) |

##### Étape 3 : Validation

- Comparer plusieurs modèles candidats
- Utiliser les critères AIC (Akaike) et BIC (Bayesian) – cf. section suivante
- Le meilleur modèle a le **AIC/BIC le plus faible**
- BIC pénalise plus les modèles complexes que AIC
- En cas de désaccord : privilégier le modèle le plus parcimonieux

Ensuite, une fois que le modèle ARIMA sélectionné a été entraîné sur un ensemble d’entraînement, on peut procéder à un diagnostic du modèle (analyse des résidus…), des prédictions, une évaluation du modèle en calculant des métriques pour mesurer l’écart entre ce qu’il prédit et les valeurs observées (ensemble de test), etc.

#### 4.2.3 Détail de fonctionnement du modèle ARIMA (si vous voulez comprendre)

Structure mathématique du modèle ARIMA(p,d,q) :
Après avoir appliqué d différenciations (∇ᵈYₜ), on modélise :
$$
\nabla^d Y_t = c + \phi_1 \nabla^d Y_{t-1} + ... + \phi_p \nabla^d Y_{t-p} + \varepsilon_t + \theta_1 \varepsilon_{t-1} + ... + \theta_q \varepsilon_{t-q}
$$

Paramètres estimés pendant l’entraînement du modèle :

* $\phi = (\phi_1, \ldots , \phi_p)$ : coefficients autorégressifs (combien chaque valeur passée influence le présent)
* $\theta = (\theta_1, \ldots , \theta_p)$ : coefficients moyenne mobile (combien chaque erreur passée influence le présent)
* $c$ : constante (dérive)
* $\sigma^2$ : variance du bruit blanc $\varepsilon_1$

La méthode d'estimation des paramètres est le maximum de vraisemblance :
L'algorithme construit la fonction de vraisemblance $L(\phi, \theta, c, \sigma^2 | \text{ données})$ qui représente la probabilité d'observer les données d'entraînement étant donnés certains paramètres. Sous l'hypothèse que les erreurs $\varepsilon_t$ suivent une distribution normale $N(0, \sigma^2)$, on peut écrire cette vraisemblance comme le produit des densités de probabilité de chaque observation conditionnellement aux précédentes. 
En pratique, on maximise le log-vraisemblance (plus facile numériquement) :
$$
\ell(\phi, \theta, c, \sigma^2) = -\frac{n}{2}\ln(2\pi) - \frac{n}{2}\ln(\sigma^2) - \frac{1}{2\sigma^2}\sum_{t=1}^{n}\varepsilon_t^2(\phi, \theta, c)
$$
L'optimisation se fait avec des algorithmes itératifs (BFGS, Newton-Raphson) qui calculent les gradients et ajustent les paramètres jusqu'à trouver le maximum. Statsmodels utilise des techniques avancées comme le filtre de Kalman pour calculer efficacement la vraisemblance, même avec des paramètres MA.

Processus de prévision :
Une fois les paramètres estimés $(\hat{\phi},\hat{\theta}, \hat{c}, \hat{\sigma^2})$ la prévision se fait récursivement :

**Horizon h=1** (un pas dans le futur) :

$$
\hat{Y}_{T+1} = \hat{c} + \hat{\phi}_1 Y_T + ... + \hat{\phi}_p Y_{T-p+1} + \hat{\theta}_1 \varepsilon_T + ... + \hat{\theta}_q \varepsilon_{T-q+1}
$$

On utilise les vraies valeurs observées jusqu'à T et les vraies erreurs (résidus du modèle).

**Horizon h=2** (deux pas dans le futur) :

$$
\hat{Y}_{T+2} = \hat{c} + \hat{\phi}_1 \hat{Y}_{T+1} + \hat{\phi}_2 Y_T + ... + \hat{\theta}_1 \varepsilon_{T+1} + \hat{\theta}_2 \varepsilon_T + \ldots
$$

**Mais attention :** $\hat{Y}_{T+1}$ est une prédiction (pas quelque chose d’observé) donc $\varepsilon_{T+1}$ est inconnu, on le remplace par son espérance = 0

**Horizon h=k :** 

* On remplace toutes les valeurs futures non observées par leurs prédictions
* On remplace toutes les erreurs futures par 0 (leur espérance)
* Donc l'incertitude augmente car on accumule les erreurs de prédiction

**Intervalles de confiance :**
La variance de l'erreur de prévision à l'horizon h est :

$$Var(\hat{Y}_{T+h} - Y_{T+h}) = \sigma^2 \times f(h, \phi, \theta)$$
où f(h, φ, θ) est une fonction croissante de h. L'intervalle de confiance à 95% est :

$$\hat{Y}_{T+h} \pm 1.96 \times \sqrt{Var(\hat{Y}_{T+h} - Y_{T+h})}$$

C'est pourquoi les intervalles s'élargissent : plus on va loin dans le futur, plus l'incertitude s'accumule.

**Critères de qualité (AIC/BIC) :**
Pendant l'entraînement, on évalue aussi :

* $AIC = -2 \log \hat{L} + 2k$ (pénalise la complexité linéairement)
* $BIC = -2 \log \hat{L} + k \log n$ (pénalise plus fortement)

où $k = p + q + 1$ (nombre de paramètres) et $\hat{L}$ est la vraisemblance maximale. Ces critères permettent de comparer différents modèles : le meilleur modèle a le plus petit AIC/BIC.

#### 4.2.4 Exemple complet : Modélisation ARIMA d'une série de ventes

##### Données

In [ ]:
# Création d’une série de vente - Marche aléatoire avec AR(1) sur les différences
np.random.seed(42)
n = 300
dates = pd.date_range(start='2019-01-01', periods=n, freq='D')

# Créer directement les différences comme un AR(1)
diff = [0]
bruit = np.random.normal(0, 3, n)
for t in range(1, n):
    diff.append(0.7 * diff[-1] + bruit[t])

# Intégrer pour obtenir la série en niveau (ARIMA au lieu de ARMA)
ventes_integrees = np.cumsum(diff) + 100  # Point de départ à 100

serie_ventes = pd.Series(ventes_integrees, index=dates, name='Ventes')


In [ ]:
# Visualisation de la série
plt.figure(figsize=(14, 6))
plt.plot(serie_ventes, linewidth=1.5, color='navy', alpha=0.7)
plt.title('Série de Ventes Journalières', fontsize=14, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Ventes', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

##### Étape 1 : Déterminer d (ordre de différenciation)

On commence par tester si la série est stationnaire :

In [ ]:
# Test de stationnarité sur la série originale
is_stationary = test_stationnarite(serie_ventes, "Ventes Originales")

La série n’est pas stationnaire : il faut donc effectuer une différenciation

In [ ]:
# Différenciation d'ordre 1
serie_diff1 = serie_ventes.diff().dropna()

# Test de stationnarité
is_stationary_diff1 = test_stationnarite(serie_diff1, "Ventes Différenciées (d=1)")

La série après une première différentiation est stationnaire. Une différenciation d’ordre 1 correspond au paramètre d=1

Visualisons ce que cela donne :

In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Série originale
axes[0].plot(serie_ventes, linewidth=1.5, color='red', alpha=0.7)
axes[0].set_title('Série Originale (Non-Stationnaire)', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Ventes', fontsize=11)
axes[0].grid(True, alpha=0.3)

# Série différenciée
axes[1].plot(serie_diff1, linewidth=1.5, color='green', alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].set_title('Série Différenciée d=1 (Stationnaire)', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_ylabel('Δ Ventes', fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

##### Étape 2 : Déterminer p et q avec ACF/PACF

Il faut déterminer à quel type de processus nous avons affaire. Les autocorrélation vont nous aider à décider :

In [ ]:
# ACF et PACF de la série différenciée
fig, axes = plt.subplots(1, 2, figsize=(16, 5))
fig.suptitle('ACF et PACF de la Série Différenciée (pour identifier p et q)', 
             fontsize=16, fontweight='bold')

# ACF
plot_acf(serie_diff1, lags=30, ax=axes[0], alpha=0.05)
axes[0].set_title('ACF', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Lag', fontsize=11)

# PACF
plot_pacf(serie_diff1, lags=30, ax=axes[1], alpha=0.05, method='ywm')
axes[1].set_title('PACF', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Lag', fontsize=11)

plt.tight_layout()
plt.show()

print("\nAnalyse :")
print("  - PACF : pic significatif au lag 1, puis coupure")
print("  - ACF : décroissance exponentielle")
print("  → Indique un processus AR(1) : p=1 et q=0")
print("\nConclusion : ARIMA(1, 1, 0) semble approprié")
print("   On testera aussi ARIMA(1, 1, 1) et ARIMA(2, 1, 0) pour comparaison")

##### Étape 3 : Entraînement et comparaison de modèles

Comme pour toute méthode de machine learning, il nous faut un ensemble d’entraînement et un ensemble de test. Le problème c’est qu’on ne peut pas mélanger les observations pour constituer les ensembles car l’ordre est important.

In [ ]:
# Division train/test (80/20)
split_index = int(len(serie_ventes) * 0.8)
train = serie_ventes[:split_index]
test = serie_ventes[split_index:]

print(f"Ensemble d'entraînement : {len(train)} observations")
print(f"Ensemble de test : {len(test)} observations")

Entraînons différents modèles ARIMA en testant plusieurs valeurs pour les paramètres, et utilisons les critères AIC/BIC pour sélectionner le meilleur modèle :

In [ ]:
# Test de plusieurs modèles candidats
modeles_candidats = [
    (1, 1, 0),
    (1, 1, 1),
    (2, 1, 0),
    (0, 1, 1),
    (2, 1, 1)
]

resultats = []

for ordre in modeles_candidats:
    try:
        # Entraînement du modèle
        modele = ARIMA(train, order=ordre)
        modele_fit = modele.fit()
        
        # Stockage des résultats
        resultats.append({
            'Ordre (p,d,q)': f"ARIMA{ordre}",
            'AIC': modele_fit.aic,
            'BIC': modele_fit.bic,
            'modele': modele_fit
        })
    except Exception as e:
        print(f"Erreur pour ARIMA{ordre}: {e}")

# Tableau comparatif
df_resultats = pd.DataFrame(resultats)[['Ordre (p,d,q)', 'AIC', 'BIC']]
df_resultats = df_resultats.sort_values('AIC')

print("\n" + "="*60)
print("COMPARAISON DES MODÈLES CANDIDATS")
print("="*60)
print(df_resultats.to_string(index=False))
print("\nLe meilleur modèle a le AIC/BIC le plus faible")

# Sélection du meilleur modèle
meilleur_idx = df_resultats.index[0]
meilleur_modele = resultats[meilleur_idx]['modele']
meilleur_ordre = resultats[meilleur_idx]['Ordre (p,d,q)']

print(f"\nMeilleur modèle : {meilleur_ordre}")

On dispose classiquement d’une methode `.summary()` qui permet d’obtenir un petit compte rendu de l’entraînement du modèle : paramètres, statistiques, tests, qualité…

In [ ]:
# Résumé du modèle
print(meilleur_modele.summary())

##### Étape 4 : analyse des résidus

On peut aussi procéder à un diagnostic « visuel » en traçant des graphes pour analyser les résidus. Ceux-ci doivent :
* avoir une distribution normale (tracer histogram + KDE ou un QQ-plot)
* avoir une moyenne à 0 (tracer les résidus et voir s’ils oscillent autour de la moyenne)
* avoir une variance uniforme (idem)
* ne pas montrer de pattern autoregressif (plot_acf / correlogramme)

In [ ]:
# Diagnostic visuel du modèle
meilleur_modele.plot_diagnostics(figsize=(16, 12))
plt.tight_layout()
plt.show()



1. Residus (standardisés) : doivent ressembler à du bruit blanc (moyenne 0, variance constante)
2. Histogram + KDE : les résidus doivent suivre une distribution normale
3. Q-Q Plot : les points doivent être alignés sur la diagonale (normalité)
4. Correlogramme (ACF) : pas de corrélation significative au sein des résidus

Il existe aussi un test pour l’autocorrélation des résidus :

In [ ]:
# Test de Ljung-Box sur les résidus (test d'autocorrélation)
residus = meilleur_modele.resid
ljung_box = acorr_ljungbox(residus, lags=10, return_df=True)

print("\nTest de Ljung-Box (autocorrélation des résidus) :")
print(ljung_box)
print("\nSi p-value > 0.05 : pas d'autocorrélation → résidus = bruit blanc")

##### Étape 5 : Prévisions (Forecast)

L’objectif de tout modèle de machine learning est de faire des prédictions. Le modèle va réaliser des prédictions de nouveaux point en prédisant à chaque fois le point suivant puis en prédisant un nouveau point à partir du précédent etc. 
Plus on prédit de point, plus l’erreur/incertitude augmente… On trace donc un intervalle de confiance des prédictions.

In [ ]:
# Prévisions sur l'ensemble de test
n_prev = len(test)
previsions = meilleur_modele.forecast(steps=n_prev)

# Calcul des intervalles de confiance
forecast_obj = meilleur_modele.get_forecast(steps=n_prev)
intervalle_confiance = forecast_obj.conf_int()

print(f"Prévisions calculées : {len(previsions)} valeurs")

In [ ]:
# Visualisation des prévisions
plt.figure(figsize=(16, 7))

# Ensemble d'entraînement
plt.plot(train.index, train.values, label='Entraînement', linewidth=2, color='navy')

# Ensemble de test (valeurs réelles)
plt.plot(test.index, test.values, label='Test (Valeurs Réelles)', 
         linewidth=2, color='green', marker='o', markersize=4)

# Prévisions
plt.plot(test.index, previsions, label='Prévisions', 
         linewidth=2, color='red', linestyle='--', marker='x', markersize=6)

# Intervalle de confiance à 95%
plt.fill_between(test.index, 
                 intervalle_confiance.iloc[:, 0], 
                 intervalle_confiance.iloc[:, 1],
                 color='red', alpha=0.2, label='Intervalle de confiance 95%')

plt.title(f'Prévisions avec {meilleur_ordre}', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Ventes', fontsize=12)
plt.legend(fontsize=11, loc='best')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

Notre prédiction est très moyenne (reporte en fait la dernière valeur observée). On verrra si on fait mieux avec des données réelles (exercice 6 suivant).

##### Étape 6 : Évaluation des performances (métriques)

Vu qu’il s’agit de valeurs numériques, les métriques usuelles dans ce cas pourront être convoquées : RMSE, MAE, MAPE…

In [ ]:
def calculer_metriques(y_true, y_pred):
    """Calcule les métriques d'évaluation"""
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    return {'RMSE': rmse, 'MAE': mae, 'MAPE': mape}

In [ ]:
# Calcul des métriques
metriques = calculer_metriques(test.values, previsions)

print("\n" + "="*60)
print("MÉTRIQUES DE PERFORMANCE SUR L'ENSEMBLE DE TEST")
print("="*60)
print(f"RMSE (Root Mean Squared Error) : {metriques['RMSE']:.4f}")
print(f"MAE (Mean Absolute Error)      : {metriques['MAE']:.4f}")
print(f"MAPE (Mean Absolute % Error)   : {metriques['MAPE']:.2f}%")
print("="*60)

print("\n💡 Interprétation :")
print(f"  - En moyenne, les prévisions s'écartent de {metriques['MAE']:.2f} unités des valeurs réelles")
print(f"  - Erreur relative moyenne : {metriques['MAPE']:.2f}%")

In [ ]:
# Analyse des erreurs de prévision
erreurs = test.values - previsions

fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Distribution des erreurs
axes[0].hist(erreurs, bins=20, color='steelblue', alpha=0.7, edgecolor='black')
axes[0].axvline(x=0, color='red', linestyle='--', linewidth=2, label='Erreur nulle')
axes[0].axvline(x=erreurs.mean(), color='orange', linestyle='--', linewidth=2, 
                label=f'Moyenne = {erreurs.mean():.2f}')
axes[0].set_title('Distribution des Erreurs de Prévision', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Erreur', fontsize=11)
axes[0].set_ylabel('Fréquence', fontsize=11)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3, axis='y')

# Erreurs au fil du temps
axes[1].plot(test.index, erreurs, marker='o', linewidth=1.5, color='crimson', alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', linewidth=2)
axes[1].set_title('Erreurs de Prévision au Fil du Temps', fontsize=13, fontweight='bold')
axes[1].set_xlabel('Date', fontsize=11)
axes[1].set_ylabel('Erreur', fontsize=11)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nStatistiques des erreurs :")
print(f"  Moyenne : {erreurs.mean():.4f}")
print(f"  Écart-type : {erreurs.std():.4f}")
print(f"  Min : {erreurs.min():.4f}")
print(f"  Max : {erreurs.max():.4f}")

##### Résumé pour se rappeler de la procédure :

1. **Visualisation** : Observer tendance, saisonnalité, comportement général (variance, etc.)
2. **Test de stationnarité** : ADF sur série originale
3. **Différenciation** : Jusqu'à obtenir stationnarité → détermine **d**
4. **ACF/PACF** : Sur série stationnaire → identifier **p** et **q**
5. **Modèles candidats** : Tester plusieurs combinaisons (p, d, q)
6. **Sélection** : Comparer AIC/BIC → choisir le meilleur
7. **Diagnostic** : Vérifier que résidus = bruit blanc (Ljung-Box, ACF résidus)
8. **Prévision** : Forecast avec intervalles de confiance + évaluation

Note :
- **d** rarement > 2
- Commencer simple : ARIMA(1,1,0), ARIMA(0,1,1)
- AIC favorise la prédiction, BIC la parcimonie. Une différence est à prendre en compte si elle est supérieure à 15/20. 
- Résidus doivent être du bruit blanc (moyenne à 0, variance constante, pas de pattern)
- Intervalles de confiance s'élargissent immanquablement avec l'horizon de prévision -> rôle de l’échantillonage. Si vous voulez une prédiction à un mois : mesures mensuelles, si vous voulez une prédiction à un jour : mesures quotidiennes

#### 4.2.5 Exercice 6 : récapitulation

Nous allons utiliser des données réelles : le dataset « Lynx » qui contient le nombre annuel de lynx piégés au Canada de 1821 à 1934 (114 observations). C'est un dataset classique en analyse de séries temporelles, notamment utilisé pour illustrer les cycles naturels de populations animales.

1. Chargez et visualisez la série, qu’observez-vous ?
2. Testez la stationnarité et différenciez si nécessaire. Comparez la série originale avec sa transformée logarithmique. Quelle est l’intérêt d’une telle transformation ? (cf. cours de l’année dernière et les problème d’hétéroscédasticité)
4. Analysez ACF et PACF pour identifier p et q
5. Entraînez plusieurs modèles ARIMA candidats
6. Comparez avec AIC/BIC et sélectionnez le meilleur
7. Diagnostiquez le modèle (résidus)
8. Réalisez des prévisions et évaluez les performances
9. Visualisez les résultats

**Modèles attendus** : ARIMA(2,1,0), ARIMA(2,1,1) ou (parfois) ARIMA(11,1,0)

In [ ]:
# Chargement du dataset Lynx

lynx_data = get_rdataset("lynx", "datasets").data
dates_lynx = pd.date_range(start='1821', periods=len(lynx_data), freq='YS')
serie_lynx = pd.Series(lynx_data['value'].values, index=dates_lynx, name='Lynx piégés')

In [ ]:
# EXERCICE 5 - VOTRE CODE

# 1. Visualisation
# plt.figure(figsize=(14, 6))
# plt.plot(serie_lynx, ...)
# ...

# 2. Test de stationnarité
# test_stationnarite(serie_lynx, "Série Lynx")
# ...

# 3. Transformation et différenciation si nécessaire
# Indice : variance croissante → transformation ... 
# serie_lynx_transformed = ...
# serie_lynx_diff = ...
# ...

# 4. ACF et PACF
# fig, axes = plt.subplots(1, 2, figsize=(16, 5))
# plot_acf(...)
# plot_pacf(...)
# ...

# 5. Entraînement de plusieurs modèles
# modeles = [(2,1,0), (1,1,0), (2,1,1), (11,1,0), (12,1,0)]
# ...

# 6. Sélection du meilleur modèle
# ...

# 7. Diagnostic
# meilleur_modele.plot_diagnostics(...)
# ...

# 8. Prévisions et évaluation
# ...

## 5. Saisonnalité et modèles SARIMA (optionnel)

Pour introduire le concept de saisonnalité, déjà observé dans l’exercice précédent, un autre exercice (que nous finirons à la fin de cette section).

### 5.1 Exercice 7 : Air Passengers

Nous allons récupérer le dataset Air Passengers, qui contient le nombre de passagers aériens mensuels à l’international sur la période (1949-1960). C’est un dataset très classique, présenté par Box & Jenkins (1976) dans leur ouvrage *Time Series Analysis*. Il fait partie des dataset disponibles dans `statsmodels`.

Analysez ce dataset :

1. Charger le dataset et exploration
2. Test de stationnarité et saisonalité :
    * tenter une différenctiation d=1
    * si cela ne fonctionne pas, appliquez une transformation logarithmique et une différenciation
    * si cela ne fonctionne toujours pas, appliquez encore une différenciation (différenciation de degré 2)
3. ACF et PACF
4. Entraînement de plusieurs modèles
5. Sélection du meilleur modèle
6. Diagnostic
7. Prévisions et évaluation

In [ ]:
# Chargement du dataset Air Passengers

air_data = get_rdataset("AirPassengers", "datasets")
serie_airpassengers = air_data.data['value']
dates = pd.date_range(start='1949-01-01', periods=len(serie_airpassengers), freq='MS')
serie_airpassengers.index = dates


In [ ]:
# EXERCICE 7 - VOTRE CODE

# 1. Visualisation
# ...

# 2. Test de stationnarité
# ...

# 3. Différenciation d=2 si nécessaire
# ...

# 4. ACF et PACF
# ...

# 5. Entraînement de plusieurs modèles
# modeles = [(1,1,0), (2,1,0), (1,1,1), (2,1,1)]
# ...

# 6. Sélection du meilleur modèle
# ...

# 7. Diagnostic
# ...

# 8. Prévisions et évaluation
# ...

Que s’est-il passé ? En fait en faisant une double différenciation alors que la période de la saisonnalité est suppérieur à 2, on n’a pas réglé le problème de la saisonnalité.

En fait la différenciation d=2 va éliminer une tendance linéaire résiduelle dans les différences. Si après d=1 il reste une légère dérive (tendance dans la variance), d=2 peut l'enlever. Mais ce n’est pas le problème ici ! Au contraire, cette différenciation supplémentaire induit les problèmes typiques d’une sur-différenciation :

- d=2 ajoute du bruit au lieu de clarifier
- la variance des résidus va augmenter
- d=2 conduit à un modèle MA induit artificiellement : différencier deux fois transforme mathématiquement un processus stationnaire en MA(1) : si $Y_t$ est un bruit blanc, alors $\nabla^2 Y_t = Y_t - 2Y_{t-1} + Y_{t-2} = \varepsilon_t - 2\varepsilon_{t-1} + \varepsilon_{t-2}$ , ce qui crée artificiellement une structure de moyenne mobile qui n'existait pas dans le processus original.
- donc les prévisions seront moins bonnes

Pour gérer les problème de saisonnalité on va mettre en œuvre un modèle SARIMA (Seasonal ARIMA)

### 5.2 Saisonnalité

#### 5.1.1 Qu'est-ce que la saisonnalité ?

La **saisonnalité** désigne des patterns réguliers qui se répètent à intervalles fixes dans une série temporelle. On la découvre généralement à l’aide d’une simple inspection visuelle des données.

##### Caractéristiques

- **Régularité** : le pattern se répète de manière prévisible
- **Période fixe** : intervalle constant (S)
- **Amplitude** : peut être constante (additive) ou variable (multiplicative)

##### Exemples de saisonnalité

| Domaine | Période | Exemple |
|---------|---------|----------|
| **Retail** | Annuelle (S=12) | Pics de ventes en décembre |
| **Tourisme** | Annuelle (S=4) | Haute saison été/hiver |
| **Énergie** | Annuelle (S=12) | Consommation chauffage/climatisation |
| **Trafic web** | Hebdomadaire (S=7) | Baisse le week-end |
| **Température** | Annuelle (S=365) | Cycles saisonniers |

##### Limites d'ARIMA classique

- ARIMA gère la **tendance** et les **corrélations à court terme**
- Mais **ne capture pas les patterns saisonniers** de manière efficace
- Il faudrait des ordres p et q très élevés → modèle complexe et peu parcimonieux (p. ex. dans l’exercice sur les Lynx nous sommes monté à ARIMA(11,0,0)

**Solution** : SARIMA, qui ajoute des composantes saisonnières spécifiques

In [ ]:
# Création d'une série avec forte saisonnalité
np.random.seed(42)
n_mois = 60  # 5 ans de données mensuelles
dates = pd.date_range(start='2019-01-01', periods=n_mois, freq='MS')

# Composantes
temps = np.arange(n_mois)
tendance = 1000 + 5 * temps  # Croissance linéaire

# Saisonnalité annuelle (période = 12 mois)
mois = np.array([d.month for d in dates])
saisonnalite = 200 * np.sin(2 * np.pi * mois / 12) + 150 * (mois == 12)  # Pic en décembre

# Bruit
bruit = np.random.normal(0, 30, n_mois)

# Série complète
ventes_saisonnieres = tendance + saisonnalite + bruit
serie_saisonniere = pd.Series(ventes_saisonnieres, index=dates, name='Ventes Mensuelles')

print(f"Série saisonnière créée : {len(serie_saisonniere)} observations")
print(f"Période : {serie_saisonniere.index[0].strftime('%Y-%m')} à {serie_saisonniere.index[-1].strftime('%Y-%m')}")

In [ ]:
# Visualisation de la série saisonnière
plt.figure(figsize=(16, 6))
plt.plot(serie_saisonniere, linewidth=2, marker='o', markersize=5, color='steelblue')
plt.title('Série de Ventes avec Saisonnalité Annuelle', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Ventes', fontsize=12)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

- Tendance croissante claire
- Patterns qui se répètent chaque année (pics réguliers)
- Saisonnalité visible à l'œil nu

#### 5.1.3 Décomposition saisonnière

La méthode `seasonal_decompose()` du module `statsmodels.tsa.seasonal` nous permet de séparer la composante saisonnière d’une série de sa composante tendancielle et du bruit. Il faut indiquer la péridode de la saison, qu’ici nous estimons annuelle, soit une période de 12 pour des données mensuelles. L’objet retourné nous permet d’accéder à ces différentes composantes :

In [ ]:
# Décomposition de la série (période = 12 pour saisonnalité annuelle)
decomposition = seasonal_decompose(serie_saisonniere, model='additive', period=12)

# Visualisation
fig, axes = plt.subplots(4, 1, figsize=(16, 14))
fig.suptitle('Décomposition Saisonnière (Période = 12 mois)', fontsize=16, fontweight='bold', y=0.995)

# Série observée
decomposition.observed.plot(ax=axes[0], color='steelblue', linewidth=2)
axes[0].set_ylabel('Observé', fontsize=11)
axes[0].set_title('Série Originale', fontsize=13, fontweight='bold')
axes[0].grid(True, alpha=0.3)

# Tendance
decomposition.trend.plot(ax=axes[1], color='green', linewidth=2)
axes[1].set_ylabel('Tendance', fontsize=11)
axes[1].set_title('Composante Tendance', fontsize=13, fontweight='bold')
axes[1].grid(True, alpha=0.3)

# Saisonnalité
decomposition.seasonal.plot(ax=axes[2], color='orange', linewidth=2)
axes[2].set_ylabel('Saisonnalité', fontsize=11)
axes[2].set_title('Composante Saisonnière (Période = 12)', fontsize=13, fontweight='bold')
axes[2].grid(True, alpha=0.3)

# Résidus
decomposition.resid.plot(ax=axes[3], color='red', linewidth=1, alpha=0.7)
axes[3].set_ylabel('Résidus', fontsize=11)
axes[3].set_xlabel('Date', fontsize=11)
axes[3].set_title('Résidus', fontsize=13, fontweight='bold')
axes[3].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nAnalyse de la décomposition :")
print(f"  - Tendance : croissance régulière de {decomposition.trend.dropna().iloc[0]:.0f} à {decomposition.trend.dropna().iloc[-1]:.0f}")
print(f"  - Saisonnalité : pattern répété chaque année (période = 12 mois)")
print(f"  - Amplitude saisonnière : ~{decomposition.seasonal.std():.0f} unités")
print(f"  - Résidus : variance résiduelle = {decomposition.resid.dropna().std():.1f}")

In [ ]:
# Visualisation du pattern saisonnier isolé (sur une année)
pattern_saisonnier = decomposition.seasonal.iloc[:12]

plt.figure(figsize=(14, 6))
plt.plot(range(1, 13), pattern_saisonnier, marker='o', markersize=10, 
         linewidth=3, color='orange', label='Pattern saisonnier')
plt.axhline(y=0, color='black', linestyle='--', alpha=0.5)
plt.xticks(range(1, 13), ['Jan', 'Fév', 'Mar', 'Avr', 'Mai', 'Jun', 
                           'Jul', 'Aoû', 'Sep', 'Oct', 'Nov', 'Déc'])
plt.title('Pattern Saisonnier (Effet de chaque mois)', fontsize=16, fontweight='bold')
plt.xlabel('Mois', fontsize=12)
plt.ylabel('Effet Saisonnier', fontsize=12)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

On constate à la lecture de ce graphique que :
- Décembre a l'effet saisonnier le plus élevé (pic de ventes)
- Février-Mars ont les valeurs les plus basses

En raison de la saisonnalité de cette observation, on observe ce pattern chaque année (répétition)

Voilà ce qu’il en est du point de vue descriptif et exploratoire, mais il nous faut établir un modèle, voyons en détail en quoi consiste un modèle SARIMA.

### 5.2 : Modèles SARIMA

#### 5.2.1 Structure SARIMA(p,d,q)(P,D,Q)[S]

**SARIMA** = **S**easonal **ARIMA**

##### Les 7 hyperparamètres

Un modèle SARIMA est noté : **SARIMA(p, d, q)(P, D, Q)[S]**

##### Partie non-saisonnière (comme ARIMA)
- **p** : ordre autorégressif
- **d** : ordre de différenciation
- **q** : ordre moyenne mobile

##### Partie saisonnière (nouvelle)
- **P** : ordre autorégressif saisonnier
- **D** : ordre de différenciation saisonnière
- **Q** : ordre moyenne mobile saisonnier
- **S** : période de la saisonnalité (12 pour mensuel annuel, 4 pour trimestriel, 7 pour hebdomadaire)

##### Équation SARIMA

La série est d'abord différenciée :
- **d** fois de manière ordinaire : $\nabla^d$
- **D** fois de manière saisonnière : $\nabla_S^D$ où $\nabla_S Y_t = Y_t - Y_{t-S}$

Ensuite, on applique les parties AR et MA (ordinaires et saisonnières) sur la série différenciée.

##### Exemples de modèles SARIMA

- **SARIMA(1,1,1)(1,1,1)[12]** : modèle complet pour données mensuelles
- **SARIMA(0,1,1)(0,1,1)[12]** : double lissage exponentiel de Holt-Winters
- **SARIMA(1,0,0)(1,0,0)[7]** : AR saisonnier pour données hebdomadaires

#### 4.2.2 Différenciation saisonnière

##### Différenciation ordinaire vs saisonnière

**Différenciation ordinaire (d)** : $\nabla Y_t = Y_t - Y_{t-1}$
- Élimine la tendance

**Différenciation saisonnière (D)** : $\nabla_S Y_t = Y_t - Y_{t-S}$
- Élimine la saisonnalité
- S = période (12 pour mensuel, 4 pour trimestriel, etc.)

On peut **combiner les deux** : $\nabla \nabla_S Y_t$ (d=1, D=1)

In [ ]:
# Illustration des différenciations

# Série originale
serie_orig = serie_saisonniere.copy()

# Différenciation ordinaire (d=1)
serie_diff_ord = serie_orig.diff().dropna()

# Différenciation saisonnière (D=1, S=12)
serie_diff_sais = serie_orig.diff(12).dropna()

# Différenciation combinée (d=1, D=1)
serie_diff_combi = serie_orig.diff(12).diff().dropna()

print("Différenciations appliquées :")
print(f"  - Série originale : {len(serie_orig)} obs")
print(f"  - Diff ordinaire (d=1) : {len(serie_diff_ord)} obs")
print(f"  - Diff saisonnière (D=1, S=12) : {len(serie_diff_sais)} obs")
print(f"  - Diff combinée (d=1, D=1) : {len(serie_diff_combi)} obs")

In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(4, 1, figsize=(16, 16))
fig.suptitle('Effet des Différentes Différenciations', fontsize=16, fontweight='bold', y=0.995)

# Série originale
axes[0].plot(serie_orig, linewidth=1.5, color='navy', alpha=0.7)
axes[0].set_title('Série Originale (Tendance + Saisonnalité)', fontsize=13, fontweight='bold')
axes[0].set_ylabel('Valeur', fontsize=11)
axes[0].grid(True, alpha=0.3)

# Diff ordinaire
axes[1].plot(serie_diff_ord, linewidth=1.5, color='green', alpha=0.7)
axes[1].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[1].set_title('Différenciation Ordinaire (d=1) - Élimine la tendance mais garde la saisonnalité', 
                  fontsize=13, fontweight='bold')
axes[1].set_ylabel('Δ Valeur', fontsize=11)
axes[1].grid(True, alpha=0.3)

# Diff saisonnière
axes[2].plot(serie_diff_sais, linewidth=1.5, color='orange', alpha=0.7)
axes[2].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[2].set_title('Différenciation Saisonnière (D=1, S=12) - Élimine la saisonnalité mais garde la tendance', 
                  fontsize=13, fontweight='bold')
axes[2].set_ylabel('Δ₁₂ Valeur', fontsize=11)
axes[2].grid(True, alpha=0.3)

# Diff combinée
axes[3].plot(serie_diff_combi, linewidth=1.5, color='red', alpha=0.7)
axes[3].axhline(y=0, color='black', linestyle='--', alpha=0.5)
axes[3].set_title('Différenciation Combinée (d=1, D=1) - Élimine tendance ET saisonnalité', 
                  fontsize=13, fontweight='bold')
axes[3].set_xlabel('Date', fontsize=11)
axes[3].set_ylabel('ΔΔ₁₂ Valeur', fontsize=11)
axes[3].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Tests de stationnarité sur les différentes séries
test_stationnarite(serie_orig, "Série Originale")
test_stationnarite(serie_diff_ord, "Diff Ordinaire (d=1)")
test_stationnarite(serie_diff_sais, "Diff Saisonnière (D=1)")
test_stationnarite(serie_diff_combi, "Diff Combinée (d=1, D=1)");

On constate que la différenciation combinée aboutie à une série non-stationnaire sur nos données synthétiques (simulées). C’est un cas typique de **sur-différenciation**.

Ça reste une observation surprenante :
- d=1 seul → STATIONNAIRE
- D=1 seul → STATIONNAIRE
- d=1 + D=1 → NON STATIONNAIRE

Sur des données SYNTHÉTIQUES avec tendance + saisonnalité ADDITIVE, une seule différenciation suffit souvent, car la combinaison d=1 + D=1 peut CRÉER une structure artificielle :
- Augmentation de la variance du bruit
- Introduction d'une composante MA non désirée
- Test ADF perturbé par la sur-différenciation

D’où les conseils suivants quand on mène une analyse saisonnière :
1. Tester d=1 seul et D=1 seul SÉPARÉMENT
2. Si l'un des deux rend stationnaire → s'arrêter là
3. Ne combiner d=1 + D=1 QUE si nécessaire (échec à la différenciation, cf. les données Air Passengers)
4. Toujours vérifier la variance avant/après différenciation

In [ ]:
# Vérification de la variance
print("\nVérification de la variance :")
print(f"  Variance série originale : {serie_orig.var():.2f}")
print(f"  Variance après d=1 : {serie_diff_ord.var():.2f}")
print(f"  Variance après D=1 : {serie_diff_sais.var():.2f}")
print(f"  Variance après d=1+D=1 : {serie_diff_combi.var():.2f}")
print("\n  → La variance AUGMENTE à nouveau avec la double différenciation !")
print("  → Signe typique de sur-différenciation")

#### 5.2.3 Identification des paramètres saisonniers (P, D, Q)

De la même manière que pour ARIMA on peut utiliser `plot_acf()` et `plot_pacf()` pour inspecter et nous guider.

##### Méthodologie

1. **Déterminer S** : identifier la période de saisonnalité
   - Observation visuelle
   - Décomposition saisonnière
   - ACF : pics significatifs aux multiples de S

2. **Déterminer D** : différenciation saisonnière
   - Test ADF sur série brute
   - Appliquer différenciation saisonnière si nécessaire
   - Généralement D = 0 ou D = 1

3. **Déterminer d** : différenciation ordinaire (comme pour ARIMA)

4. **Déterminer P et Q** : analyser ACF et PACF aux **lags saisonniers** (S, 2S, 3S...)
   - Pics aux multiples de S sur ACF → suggère Q
   - Pics aux multiples de S sur PACF → suggère P

5. **Déterminer p et q** : analyser ACF et PACF aux **lags courts** (comme ARIMA)

#### 5.2.4 Entraînement d'un modèle SARIMA

Comme on ne peut pas utiliser nos données synthétiques, le code suivant est donné à titre de référence (inutile de l’exécuter). 
Vous pourrez le tester dans le dernier exercice de ce notebook, où nous reprendrons les données Air Passengers

##### Détermination des ensembles de tests/entrainements

```python
# Division train/test
split_idx = int(len(serie_saisonniere) * 0.8)
train_sais = serie_saisonniere[:split_idx]
test_sais = serie_saisonniere[split_idx:]
```

##### Entraînement du modèles (en fait plusieurs modèles candidats)

```python
# Entraînement de plusieurs modèles SARIMA candidats
modeles_sarima = [
    # (p,d,q), (P,D,Q,S)
    ((1,1,1), (1,1,1,12)),
    ((0,1,1), (0,1,1,12)),
    ((1,1,0), (1,1,0,12)),
    ((0,1,0), (1,1,1,12)),
    ((1,1,1), (1,1,0,12)),
]

resultats_sarima = []

print("Entraînement des modèles SARIMA...\n")

for ordre, ordre_sais in modeles_sarima:
    try:
        # Entraînement
        modele = SARIMAX(train_sais, order=ordre, seasonal_order=ordre_sais, 
                        enforce_stationarity=False, enforce_invertibility=False)
        modele_fit = modele.fit(disp=False)
        
        # Stockage
        resultats_sarima.append({
            'Ordre': f"SARIMA{ordre}{ordre_sais}",
            'AIC': modele_fit.aic,
            'BIC': modele_fit.bic,
            'modele': modele_fit
        })
        print(f"SARIMA{ordre}{ordre_sais} - AIC: {modele_fit.aic:.2f}, BIC: {modele_fit.bic:.2f}")
    except Exception as e:
        print(f"SARIMA{ordre}{ordre_sais} - Erreur: {str(e)[:50]}")

# Tableau comparatif
df_sarima = pd.DataFrame(resultats_sarima)[['Ordre', 'AIC', 'BIC']].sort_values('AIC')

print("\n" + "="*70)
print("COMPARAISON DES MODÈLES SARIMA")
print("="*70)
print(df_sarima.to_string(index=False))

# Sélection du meilleur
meilleur_idx_sais = df_sarima.index[0]
meilleur_sarima = resultats_sarima[meilleur_idx_sais]['modele']
meilleur_ordre_sais = resultats_sarima[meilleur_idx_sais]['Ordre']

print(f"\nMeilleur modèle : {meilleur_ordre_sais}")

# Résumé du meilleur modèle
print(meilleur_sarima.summary())
```

#### 5.2.5 Diagnostic du modèle SARIMA

```python
# Diagnostic visuel
meilleur_sarima.plot_diagnostics(figsize=(16, 12))
plt.tight_layout()
plt.show()

print("\nVérifications :")
print("  1. Résidus doivent osciller autour de 0")
print("  2. Histogramme des résidus doit être proche d'une normale")
print("  3. Q-Q plot : points alignés sur la diagonale")
print("  4. ACF des résidus : pas de corrélation significative")

# Test de Ljung-Box sur les résidus
residus_sarima = meilleur_sarima.resid
ljung_box_sarima = acorr_ljungbox(residus_sarima, lags=20, return_df=True)

print("Test de Ljung-Box (Autocorrélation des résidus) :")
print(ljung_box_sarima.head(10))
print("\np-value > 0.05 pour la plupart des lags → résidus = bruit blanc ")
```

#### 5.2.6 Prévisions (forecast) avec SARIMA

```python
# Prévisions sur l'ensemble de test
n_prev_sais = len(test_sais)
previsions_sarima = meilleur_sarima.forecast(steps=n_prev_sais)

# Intervalles de confiance
forecast_sarima = meilleur_sarima.get_forecast(steps=n_prev_sais)
ic_sarima = forecast_sarima.conf_int()

print(f"Prévisions SARIMA calculées : {len(previsions_sarima)} valeurs")

# Visualisation des prévisions
plt.figure(figsize=(16, 8))

# Données d'entraînement
plt.plot(train_sais.index, train_sais.values, label='Entraînement', 
         linewidth=2, color='navy')

# Données de test (valeurs réelles)
plt.plot(test_sais.index, test_sais.values, label='Test (Réel)', 
         linewidth=2.5, color='green', marker='o', markersize=6)

# Prévisions
plt.plot(test_sais.index, previsions_sarima, label='Prévisions SARIMA', 
         linewidth=2.5, color='red', linestyle='--', marker='x', markersize=8)

# Intervalle de confiance à 95%
plt.fill_between(test_sais.index, 
                 ic_sarima.iloc[:, 0], 
                 ic_sarima.iloc[:, 1],
                 color='red', alpha=0.2, label='Intervalle de confiance 95%')

plt.title(f'Prévisions avec {meilleur_ordre_sais}', fontsize=16, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.ylabel('Ventes', fontsize=12)
plt.legend(fontsize=11, loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("\nObservation :")
print("  - Les prévisions SARIMA capturent à la fois la tendance ET la saisonnalité")
print("  - Le pattern saisonnier est bien reproduit dans les prévisions")

# Évaluation des performances
metriques_sarima = calculer_metriques(test_sais.values, previsions_sarima)

print("\n" + "="*60)
print("MÉTRIQUES DE PERFORMANCE")
print("="*60)
print(f"RMSE : {metriques_sarima['RMSE']:.4f}")
print(f"MAE  : {metriques_sarima['MAE']:.4f}")
print(f"MAPE : {metriques_sarima['MAPE']:.2f}%")
print("="*60)

# Comparaison avec un modèle ARIMA simple (sans composante saisonnière)
print("\nComparaison ARIMA vs SARIMA :")
try:
    from statsmodels.tsa.arima.model import ARIMA
    arima_simple = ARIMA(train_sais, order=(1,1,1)).fit()
    prev_arima_simple = arima_simple.forecast(steps=len(test_sais))
    metriques_arima = calculer_metriques(test_sais.values, prev_arima_simple)
    
    print(f"\nARIMA(1,1,1) sans saisonnalité :")
    print(f"  RMSE : {metriques_arima['RMSE']:.4f}")
    print(f"  MAE  : {metriques_arima['MAE']:.4f}")
    print(f"  MAPE : {metriques_arima['MAPE']:.2f}%")
    
    print(f"\n{meilleur_ordre_sais} :")
    print(f"  RMSE : {metriques_sarima['RMSE']:.4f}")
    print(f"  MAE  : {metriques_sarima['MAE']:.4f}")
    print(f"  MAPE : {metriques_sarima['MAPE']:.2f}%")
    
    amelioration = ((metriques_arima['RMSE'] - metriques_sarima['RMSE']) / metriques_arima['RMSE']) * 100
    print(f"\nSARIMA améliore la RMSE de {amelioration:.1f}% par rapport à ARIMA simple")
except:
    print("  Comparaison non disponible")
```

### EXERCICE 6 : SARIMA - Dataset AirPassengers

Pour conclure ce notebook et mettre en application, nous allons mettre en œuvre SARIMA sur une série avec saisonnalité et comparer avec ARIMA

Reprenons le dataset **AirPassengers** (1949-1960, mensuel) que nous avons laissé de côté dans l'exercice 5 car il nécessite SARIMA, pas juste ARIMA.

**Rappel du problème** : 
- Tendance croissante
- Variance croissante
- Mais **Saisonnalité mensuelle (S=12)**
- Après log + diff(1), la série n'était pas totalement stationnaire (p ≈ 0.07)
  → ARIMA seul ne suffit pas, il faut capturer la saisonnalité !

**Consigne** :
1. Chargez et visualisez la série (avec transformation log)
2. Décomposition saisonnière pour confirmer S=12
3. Testez la stationnarité avec différenciations ordinaire ET saisonnière
4. Analysez ACF et PACF pour identifier les ordres
5. Entraînez plusieurs modèles SARIMA(p,d,q)(P,D,Q)[12]
6. **Comparez ARIMA vs SARIMA** pour montrer l'amélioration
7. Diagnostiquez le meilleur modèle
8. Réalisez des prévisions

**Modèles attendus** : SARIMA(0,1,1)(0,1,1)[12] ou SARIMA(0,1,1)(1,1,0)[12]

In [ ]:
# Chargement du dataset Air Passengers

air_data = get_rdataset("AirPassengers", "datasets")
serie_airpassengers = air_data.data['value']
dates = pd.date_range(start='1949-01-01', periods=len(serie_airpassengers), freq='MS')
serie_airpassengers.index = dates

In [ ]:
# EXERCICE 6 - VOTRE CODE

# 1. Visualisation et transformation log
# serie_air_log = np.log(serie_airpassengers)
# plt.figure(...)
# ...

# 2. Décomposition saisonnière
# decomp_air = seasonal_decompose(serie_air_log, model='additive', period=12)
# ...

# 3. S = 12 (mensuel)

# 4. Tests de stationnarité et différenciations
# test_stationnarite(serie_air_log, "Log")
# serie_air_diff_ord = serie_air_log.diff().dropna()
# serie_air_diff_sais = serie_air_log.diff(12).dropna()
# serie_air_diff_combi = serie_air_log.diff(12).diff().dropna()
# ...

# 5. ACF et PACF
# fig, axes = plt.subplots(1, 2, figsize=(16, 5))
# plot_acf(serie_air_diff_combi, lags=40, ...)
# plot_pacf(serie_air_diff_combi, lags=40, ...)
# ...

# 6-7. Entraînement SARIMA et comparaison avec ARIMA
# modeles_sarima = [((0,1,1), (0,1,1,12)), ((0,1,1), (1,1,0,12)), ...]
# modeles_arima = [(0,1,1), (1,1,1), ...]
# Comparer les AIC/BIC
# ...

# 8. Diagnostic
# meilleur_modele.plot_diagnostics(...)
# ...

# 9. Prévisions
# N'oubliez pas de reconvertir avec np.exp() !
# ...

In [ ]:
# 2. Décomposition saisonnière (S=12 pour mensuel)
decomp_air = seasonal_decompose(serie_air_log, model='additive', period=12)

fig, axes = plt.subplots(4, 1, figsize=(14, 12))
fig.suptitle('Décomposition Saisonnière (Période = 12 mois)', fontsize=16, fontweight='bold')

decomp_air.observed.plot(ax=axes[0], color='steelblue', linewidth=2)
axes[0].set_ylabel('Observé', fontsize=11)
axes[0].set_title('Série Originale (log)', fontsize=12)

decomp_air.trend.plot(ax=axes[1], color='green', linewidth=2)
axes[1].set_ylabel('Tendance', fontsize=11)
axes[1].set_title('Tendance (croissance du trafic aérien)', fontsize=12)

decomp_air.seasonal.plot(ax=axes[2], color='orange', linewidth=2)
axes[2].set_ylabel('Saisonnalité', fontsize=11)
axes[2].set_title('Saisonnalité (pic en été, creux en hiver)', fontsize=12)

decomp_air.resid.plot(ax=axes[3], color='red', linewidth=1)
axes[3].set_ylabel('Résidus', fontsize=11)
axes[3].set_xlabel('Mois', fontsize=11)
axes[3].set_title('Résidus', fontsize=12)

plt.tight_layout()
plt.show()

conclusions = '''
Période identifiée : S = 12 (mensuelle/annuelle)
La décomposition montre clairement :
- Tendance : croissance linéaire (en log)
- Saisonnalité : pattern annuel répété
- Résidus : relativement faibles et aléatoires
'''

print(conclusions)

In [ ]:
# 6-7. Entraînement et comparaison ARIMA vs SARIMA
split_air = int(len(serie_air_log) * 0.8)
train_air = serie_air_log[:split_air]
test_air = serie_air_log[split_air:]

print(f"Train : {len(train_air)} mois ({train_air.index[0].strftime('%Y-%m')} à {train_air.index[-1].strftime('%Y-%m')})")
print(f"Test  : {len(test_air)} mois ({test_air.index[0].strftime('%Y-%m')} à {test_air.index[-1].strftime('%Y-%m')})")

print("\n" + "="*70)
print("COMPARAISON : ARIMA vs SARIMA")
print("="*70)

# Modèles SARIMA (avec composante saisonnière)
print("\nModèles SARIMA (p,d,q)(P,D,Q)[12] :")
modeles_sarima = [
    ((0,1,1), (0,1,1,12)),  # Modèle classique Box-Jenkins
    ((0,1,1), (1,1,0,12)),  # Alternative
    ((1,1,1), (0,1,1,12)),  # Avec AR court
    ((0,1,1), (1,1,1,12)),  # Combinaison
    ((1,1,0), (0,1,1,12)),  # AR court, MA saisonnier
]

resultats_sarima = []
for ordre, ordre_sais in modeles_sarima:
    try:
        modele = SARIMAX(train_air, order=ordre, seasonal_order=ordre_sais,
                        enforce_stationarity=False, enforce_invertibility=False)
        modele_fit = modele.fit(disp=False)
        resultats_sarima.append({
            'Ordre': f"SARIMA{ordre}{ordre_sais}",
            'Type': 'SARIMA',
            'AIC': modele_fit.aic,
            'BIC': modele_fit.bic,
            'modele': modele_fit
        })
        print(f"  SARIMA{ordre}{ordre_sais} - AIC: {modele_fit.aic:.2f}")
    except Exception as e:
        print(f"  SARIMA{ordre}{ordre_sais} - Erreur")

# Modèles ARIMA (SANS composante saisonnière - pour comparaison)
print("\nModèles ARIMA (p,d,q) SANS saisonnalité (pour comparaison) :")
modeles_arima = [
    (0,1,1),  # MA(1)
    (1,1,1),  # ARMA(1,1)
    (2,1,1),  # ARMA(2,1)
    (1,1,0),  # AR(1)
]

resultats_arima = []
for ordre in modeles_arima:
    try:
        modele = ARIMA(train_air, order=ordre)
        modele_fit = modele.fit()
        resultats_arima.append({
            'Ordre': f"ARIMA{ordre}",
            'Type': 'ARIMA',
            'AIC': modele_fit.aic,
            'BIC': modele_fit.bic,
            'modele': modele_fit
        })
        print(f"  ARIMA{ordre} - AIC: {modele_fit.aic:.2f}")
    except Exception as e:
        print(f"  ARIMA{ordre} - Erreur")

# Combiner les résultats
tous_resultats = resultats_sarima + resultats_arima
df_comparaison = pd.DataFrame(tous_resultats)[['Type', 'Ordre', 'AIC', 'BIC']].sort_values('AIC')

print("\n" + "="*70)
print("COMPARAISON GLOBALE (classés par AIC) :")
print("="*70)
print(df_comparaison.to_string(index=False))

meilleur_global = tous_resultats[df_comparaison.index[0]]
meilleur_modele_air = meilleur_global['modele']
meilleur_ordre_air = meilleur_global['Ordre']
meilleur_type = meilleur_global['Type']

print(f"\n" + "="*70)
print(f"MEILLEUR MODÈLE : {meilleur_ordre_air} ({meilleur_type})")
print("="*70)

# Analyse de la différence
meilleur_arima = df_comparaison[df_comparaison['Type'] == 'ARIMA'].iloc[0]
meilleur_sarima = df_comparaison[df_comparaison['Type'] == 'SARIMA'].iloc[0]

print(f"\nANALYSE :")
print(f"  Meilleur ARIMA : {meilleur_arima['Ordre']} (AIC = {meilleur_arima['AIC']:.2f})")
print(f"  Meilleur SARIMA : {meilleur_sarima['Ordre']} (AIC = {meilleur_sarima['AIC']:.2f})")
print(f"  Amélioration AIC : {meilleur_arima['AIC'] - meilleur_sarima['AIC']:.2f} points")
print(f"\nCONCLUSION :")
print(f"  SARIMA est NETTEMENT MEILLEUR qu'ARIMA pour cette série !")
print(f"  La composante saisonnière (P,D,Q)[12] est ESSENTIELLE.")
print(f"  C'est pourquoi nous ne pouvions pas modéliser AirPassengers")
print(f"  correctement dans l'exercice 5 (ARIMA seul).")

## Pour aller plus loin

### Modèles, bibliothèques

- **SARIMAX** : ajoute des variables exogènes
- **Darts** : une bibliothèque pour les séries temporelles
```python
!pip install darts --quiet

import darts
print(f"Darts version: {darts.__version__}")
```
- **Prophet** (Facebook) : gestion automatique de la saisonnalité
- **LSTM/GRU** : réseaux de neurones récurrents pour séries temporelles
- **Transformer models** : attention mechanism pour séries temporelles

### Où trouver des séries temporelles :

- API comme Yahoo! Finance `pip install yfinance` pour des données boursières
- API comme [Google Trends](https://trends.google.com/)  `pip install pytrends` (popularité des recherches Google dans le temps), par exemple essayez d’analyser et de mettre en relation des recherches comme `'hot weather'` et `'ice cream'`
- API données météo par exemple, etc.
- les sites déjà conseillés mettant à disposition des données  ([Kaggle](https://kaggle.com/), [data.gouv.fr](https://www.data.gouv.fr/), etc.)